In [1]:
# Multiple AGN (MAGN) Catalog DR1 - Quasar Pair Catalog Formatting
# Author: R. W. Pfeifle
# Legacy Notebook Created: 10 Sept. 2020
# Quasar Pairs Notebook Created: 10 Aug. 2023
# Last Revision Date: 26 September 2023

# Purpose:
# 
# 

Update 16 Sept. 2020
---


Here is how I want to construct the catalog now and moving forward:

- There will be TWO catalogs, which are permutations of a single list. 
    - One catalog will have only one system per line, and we will have ONLY pairs in this one. This will make calculations FAR easier for individual systems (such as redshift or velocity differences), as this circumvents the problems of not being able to do row-wise operations.
    - The other catalog will be a copy of the first, with the exception that this catalog will have one line per system component, i.e. two lines per pair. This will be the catalog used for matching to other catalogs. Thus this one is a derivative of the first.
    - This same setup will be used for systems of triples, quadruples, quintuples, etc. Different order multiplets will be in different catalogs. This will make things easier to comprehend and will eliminate needed a million columns within only a single table that are empty for any system with fewer than the max number. 


--------
Summary:
--------
The purpose of this catalog is to have an exhaustive list of all known multiple
AGNs from the literature. A multiple AGN is defined as the manifestation of two
or more AGNs at some wavelength that are associated with each other due to some
astrophysical process. This astrophysical process can be:

- Multiple AGNs in a galaxy interaction, merger, or post-merger
- Multiple AGNs that are gravitationally bound, e.g. binary AGNs
- The appearance of multiple AGNs due to gravitational lensing (e.g. an Einstein cross)

This catalog should be *exhaustive*, so include not only confirmed systems but
also candidate systems. These systems should be everything that indicates or
potentially indicates the presence or apparent presence of more than one AGN
associated with a system (e.g. multiple X-ray/radio component, doubling of
spectroscopic lines, astrometric jitter, etc.)

----------------------
WMAGN catalog columns:
----------------------

- System type (Dual/triple AGN, binary AGN, or gravitational lens)

- Discovery method (e.g., multiple X-ray/radio components, spectroscopic, astrometric, etc.)

- Literature name (e.g. SDSS J084905.51+111447.2)

- Component (e.g. Gal1/Gal2/Gal3, a/b/c, 1/2/3, etc)
- One row for each component; components share same unique system name

- Unique, standardized system name
    - NED, Simbad, or IAU name preferred

- Coordinates
    - Must be in ICRS system. If listed in FK5/FK6, convert to ICRS (e.g. with AstroPy)
    - Equinox J2000.0
    - Native epoch (i.e. not corrected for "proper motion")
    - Degrees, full precision (i.e. 0.00001 deg = 0.036 arcsec)
    - If coordinates not available for each component individually, just repeat the listed system coordinates for each source.

- Coordinate epoch (e.g. 2015.5 for Gaia DR2)
    - Leave blank if not determinable

- Coordinate wavelength/frequency
    - Leave blank if not determinable

- Coordinate source (e.g. SDSS, 2MASS, VLBA, Chandra, etc)

- Redshift
    - Leave blank if not determinable

- Redshift type (spec/photo)
    - Leave blank if not determinable

- Brightness band
    - E.g., Johnson V, SDSS r, Gaia G, WISE W1, radio X-band, etc.

- Brightness numerical value
    - Magnitude or flux density

- Brightness type
    - E.g., VEGAMAG, ABMAG/STMAG, Jy

- BibCode(s), e.g. 2019ApJ...883..167P,2019ApJ...887...90L
    - Ordered by date

- NED hyperlink

- Simbad hyperlink

- Confidence level
    - Up to you how you want to qualify this. E.g., "potential", "likely", "confirmed"
    - Preferably numeric so catalog can be sorted. E.g. 1=potential, 2=likely, 3=confirmed.

- Essential object notes


In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
# # This function is for converting namings in the following format to coordinates. This is useful for later \
# # catalogs 2010-2020

# def name_to_coords(df,dfcol):
#     # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
#     # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
#     df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
#     df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
#     df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
#     df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#     df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#     df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#     return


In [4]:
# def name_to_coords(df):
#     # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
#     # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
#     df['Coordinates'] = df['SDSS'].str.slice(start=1) # Stripping the J
#     df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
#     df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
#     df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#     df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#     df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#     return


In [5]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [6]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2




In [7]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib_triples(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA3 and Dec3 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA3_deg']*u.degree, dec=t2['Dec3_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_3, idx2_3, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_3)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_3)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_3 = pd.concat([(t1.iloc[idx1_3]),(t2.iloc[idx2_3])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_3 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2, tunique_3, tmatches_3, idx1_3, idx2_3





In [8]:
# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,smith2012,5)

# # Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
# for index, row in tmatches.iterrows():
#     if row['Table_flag']!='Table2':
#         tmatches.at[index, 'Paper(s)'] += ' ; Smith+2012'
#         tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJ...752...63S' 
#         tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/752/1/63'

# # Now clipping out all Smith+2010 rows from the matches table
# tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# # Concatenating everything together to generate a master table here
# the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
# the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011d,3)

# #len(tmatches)

# # This finds 16 matches, which is exactly what we'd expect (there are 16 doubles in the table from Fu+2011)

# for i, j in zip(idx1, idx2):
#     #print("i:", i, "j:", j)
#     #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
#     #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
#     the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
#     the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
#     the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
#     the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
#     the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
#     the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# for index, row in the_whills.iterrows():
#     if index in idx1:
#         the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
#         the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
#         the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
#         the_whills.at[index, 'Notes'] += ' Companion(s) within 3 arcseconds.'



In [9]:
# Here we're loading in the catalog(s) from Hennawi+2006:

# Table 2 are binary quasars discovered from follow-up observations (sep 3 arcsec < delta theta < 60 arcsec)
hennawi2006_t2 = (Table.read('Tables/Hennawi2006/table2.dat', readme = 'Tables/Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()

# update on 6 December 2023
# here we're replacing the selection methods in t2 with what we're going to use in the MAC
hennawi2006_t2.loc[hennawi2006_t2.Sample == 'lens', 'Sample'] = 'Optical Imaging'
hennawi2006_t2.loc[hennawi2006_t2.Sample == 'photo', 'Sample'] = 'Optical Colors'
hennawi2006_t2.loc[hennawi2006_t2.Sample == '{chi}^2^', 'Sample'] = 'Optical Colors'

# Table 3 are binary quasars discovered via the lens selection (sep <3 arcsec)
hennawi2006_t3 = (Table.read('Tables/Hennawi2006/table3.dat', readme = 'Tables/Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
hennawi2006_t3['Sample'] = 'Optical Imaging' #'Lens Selection'
# Update on 6 Dec 2023: changing Lens Selection --> Optical Imaging

# Table 4 are binary quasars discovered overlapping plates from SDSS + 2DQZ quasars
hennawi2006_t4 = (Table.read('Tables/Hennawi2006/table4.dat', readme = 'Tables/Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
hennawi2006_t4['Sample'] = 'Fiber Spectroscopy / Optical Spectroscopy' #'Overlapping Plates'
# Update on 6 Dec 2023: changing Overlapping Plates --> Fiber Spectroscopy / Optical Spectroscopy

# Table 8 contains projected quasar pairs discovered via overlapping plates and the SDSS +2QZ catalog. 
# Some of these are not actually projected pairs (after inspection of the table), and therefore we will need to \
# include this table (and cut it down to only the relevant listings).
hennawi2006_t8 = (Table.read('Tables/Hennawi2006/table8.dat', readme = 'Tables/Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
#hennawi2006_t8['dV'] = 'None' # comment this out on Dec 6 2023
# Need to overwrite the NaN values in the 'Sample' column so that it reflects the fact that these come from \
# Table 8 'Project Pairs'
# Update on 6 Dec 2023: changing Projected Pairs --> Fiber Spectroscopy / Optical Spectroscopy
hennawi2006_t8['Sample'] = 'Fiber Spectroscopy / Optical Spectroscopy' #"Projected Pairs"

# Table 9 contains projected pairs discovered via overlapping plates. No need to include in catalog or load in here.
#hennawi2006_t9 = (Table.read('Tables/Hennawi2006/table9.dat', readme = 'Tables/Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()

# Table 10 has only quasar-star pairs. No need to include in catalog or load in here.


# MAGNITUDES QUOTED IN HENNAWI+ TABLES ARE EXTINCTION CORRECTED!

In [10]:
#hennawi2006_t3

In [11]:
# All of the Hennawi+2006 catalogs are structured the same, so we may as well combine them all here, and then \
# perform any column operations or column renaming that is required.

# Now concatenating Tables 2, 3 and 4 of Hennawi+2006 catalogs 
frames = [hennawi2006_t2,hennawi2006_t3,hennawi2006_t4,hennawi2006_t8]
hennawi2006 = (pd.concat(frames)).reset_index(drop=True) # Now we have a master catalog for the Hennawi+ tables.

# Now creating appropriate RA and Dec coordinate columns:
hennawi2006['RA1'] = hennawi2006['RA1.h'].astype(str) + ':' + hennawi2006['RA1.m'].astype(str) + ':' + hennawi2006['RA1.s'].astype(str)
hennawi2006['Dec1'] = hennawi2006['DE1.-'].astype(str) + hennawi2006['DE1.d'].astype(str) + ':' + hennawi2006['DE1.m'].astype(str) + ':' + hennawi2006['DE1.s'].astype(str)

hennawi2006['RA2'] = hennawi2006['RA2.h'].astype(str) + ':' + hennawi2006['RA2.m'].astype(str) + ':' + hennawi2006['RA2.s'].astype(str)
hennawi2006['Dec2'] = hennawi2006['DE2.-'].astype(str) + hennawi2006['DE2.d'].astype(str) + ':' + hennawi2006['DE2.m'].astype(str) + ':' + hennawi2006['DE2.s'].astype(str)

# Now dropping the original 14 columns used for RA and Dec...
hennawi2006.drop(labels=['RA1.h','RA1.m','RA1.s','DE1.-','DE1.d','DE1.m','DE1.s','RA2.h','RA2.m','RA2.s','DE2.-','DE2.d','DE2.m','DE2.s'], axis=1, inplace=True)

# Dropping irrelevant magnitude columns (we likely won't be using these anyhow...)
hennawi2006.drop(labels=['umag1','rmag1','imag1','zmag1','umag2','rmag2','imag2','zmag2'], axis=1, inplace=True)

# Substituting in information for SDSS J1116+4118A/B from Ellison+2007 here:
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'z1'] = 2.982
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'z2'] = 3.007

# Adding informative columns about the coordinates
hennawi2006['Equinox1'] = "J2000"
hennawi2006['Coordinate_waveband1'] = "Optical"
hennawi2006['Coordinate_Source1'] = "SDSS"

hennawi2006['Equinox2'] = "J2000"
hennawi2006['Coordinate_waveband2'] = "Optical"
hennawi2006['Coordinate_Source2'] = "SDSS"

hennawi2006['System Type'] = 'Binary Quasar'
hennawi2006['z1_type'] = 'spec'
hennawi2006['z2_type'] = 'spec'

hennawi2006['Brightness_band1'] = 'SDSS g PSF ext-corr'
hennawi2006['Brightness_band2'] = 'SDSS g PSF ext-corr'

hennawi2006['Brightness_type1'] = 'asinh'
hennawi2006['Brightness_type2'] = 'asinh'

# Calculating here redshift difference (delta_z) and velocity differences dV
# dV is defined as: dv = (2.99e+5)*((1+z1)**2 - (1+z2)**2)/((1+z1)**2+(1+z2)**2)
# These come from  the work in Tytler+2009 for velocity differences and LOS distances derived from velocity differences

#v=H(z)*d, [H(z)/H0]**2 = ((1+z)**3)*Om0 + OmV
#    Therefore: H(z) = H0*np.sqrt(((1+z)**3)*Om0 + OmV)
#d = v/H(z)

#hennawi2006['delta_z'] = hennawi2006['z1'] - hennawi2006['z2']
hennawi2006['dV_rwp'] = (2.99e+5)*((1+hennawi2006['z1'])**2 - (1+hennawi2006['z2'])**2)/((1+hennawi2006['z1'])**2+(1+hennawi2006['z2'])**2)
# Defining it as dV_rwp for now since Hennawi+ also calculated this, and the values are discrepant with mine.

# The following will also calculate the LOS distance between the two objects
#hennawi2006['LOS_dist(Mpc)'] = np.abs(hennawi2006['dV_new'])/(70*np.sqrt(((1+hennawi2006['z1'])**3)*0.3 + 0.7))

# Now calculating the projected separation in kpc from the arcsec separation...
#hennawi2006['r_kpc'] = hennawi2006['r']/(0.72) # This was a check on Hennawi's calculations...
#hennawi2006['Sep(kpc)'] = hennawi2006['theta']*((cosmo.arcsec_per_kpc_proper(hennawi2006['z1']))**(-1))
# The separation is based upon the redshift of the first quasar in each pair, but the difference should be \
# minimal if z2 was used instead.

hennawi2006['Sep'] = hennawi2006['theta']

# Now adding in a few columns to denote the confirmation method, author name+year, bibcode, and doi link
hennawi2006['Selection Method'] = hennawi2006['Sample']
hennawi2006['Confirmation Method'] = "Slit Optical Spectroscopy" # added 'slit' on 6 Dec 2023
hennawi2006['Paper(s)'] = "Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'Paper(s)'] = "Hennawi+2006 ; Ellison+2007"
hennawi2006['BibCode(s)'] = "2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'BibCode(s)'] = "2006AJ....131....1H ; 2007MNRAS.378..801E "
hennawi2006['DOI(s)'] = "https://doi.org/10.1086/498235"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'DOI(s)'] = "https://doi.org/10.1086/498235 ; https://doi.org/10.1111/j.1365-2966.2007.11847.x"

# Taking into account the overlap between Hennawi+2006 and Miller+2004
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'Paper(s)'] = "Miller+2004 ; Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'BibCode(s)'] = "2004MNRAS.348..395M ; 2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2004.07303.x ; https://doi.org/10.1086/498235"

# Taking into account the overlap between Pindor+2006 and Hennawi+2006: Pindor+2006 identified the dual quasar \
# SDSSJ1600+0000A. (Pindor is also an author on Hennawi's paper)
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'Paper(s)'] = "Pindor+2006 ; Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'BibCode(s)'] = "2006AJ....131...41P ; 2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'DOI(s)'] = "https://doi.org/10.1086/497965 ; https://doi.org/10.1086/498235"

hennawi2006['PairID'] = hennawi2006.index

# Now removing all systems which do not meet our cut in velocity difference of ≤2000 km s**-1
hennawi2006 = (hennawi2006[np.abs(hennawi2006['dV_rwp'])<=2000]).reset_index(drop=True)
# (This leaves 284 pairs)

#hennawi2006.sort_values(by='Name1', inplace=True)
# Here we're dropping the obvious duplicates where the A and B components are not mixed up between tables
hennawi2006 = hennawi2006.drop_duplicates(subset=['Name1','Name2','z1','z2','theta','RA1','Dec1','RA2','Dec2'])
hennawi2006 = hennawi2006.drop_duplicates(subset=['Name1','Name2']) #removing duplicates that have correct components only

# Here we need to do something clever, because Hennawi+2006 mixed up the A and B designations between different \
# tables I'm going to make two new columns: sum of z's and sum of gmag's
hennawi2006['summedz'] = hennawi2006['z1']+hennawi2006['z2']
hennawi2006['summedgmag'] = hennawi2006['gmag1']+hennawi2006['gmag2']

# Now this allows me to circumvent the issue of mixed up redshift and gmag columns. Nothing I can do about the \
# mixed up names, but NONE of the rows should have duplicate summed redshifts AND gmags AND identical separations \
# unless the rows are in fact the same pair.
hennawi2006 = hennawi2006.drop_duplicates(subset=['summedz','summedgmag','theta'])

# Now dropping any remaining irrelevant columns:
hennawi2006.drop(labels=['Sample','dV','chi2','PairID','summedz','summedgmag'], axis=1, inplace=True)

coordconvert = SkyCoord(ra = hennawi2006['RA1'], dec = hennawi2006['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2006['RA1_deg'] = coordconvert.ra.degree
hennawi2006['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = hennawi2006['RA2'], dec = hennawi2006['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2006['RA2_deg'] = coordconvert.ra.degree
hennawi2006['Dec2_deg'] = coordconvert.dec.degree

hennawi2006.reset_index(drop=True, inplace=True)
               

In [12]:
hennawi2006 = hennawi2006[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type',\
                           'z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
                           'Equinox1','Coordinate_waveband1','Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','gmag1','Brightness_band1',\
                           'Brightness_type1','gmag2','Brightness_band2','Brightness_type2','theta','r',\
                           'dV_rwp','Paper(s)','BibCode(s)','DOI(s)']]

hennawi2006.drop(labels=['r'], axis=1, inplace=True) # Dropping this column, since we separately calculate it in 'Sep(kpc)'

# Here we're renaming the columns:
# dV_rwp --> dV
# gmag1 --> Brightness1 and gmag2 --> Brightness2
hennawi2006.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type',\
                           'z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
                           'Equinox1','Coordinate_waveband1','Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','Brightness1','Brightness_band1',\
                           'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep',\
                           'dV','Paper(s)','BibCode(s)','DOI(s)']


In [13]:
hennawi2006

,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Paper(s),BibCode(s),DOI(s)
0,Binary Quasar,SDSSJ0048-1051A,SDSSJ0048-1051B,Optical Colors,Slit Optical Spectroscopy,1.555,spec,1.560,spec,0:48:0.77,-10:51:48.6,12.003208,-10.863500,0:48:0.96,-10:51:46.2,12.004000,-10.862833,J2000,Optical,SDSS,J2000,Optical,SDSS,20.52,SDSS g PSF ext-corr,asinh,20.05,SDSS g PSF ext-corr,asinh,3.6,-584.554671,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
1,Binary Quasar,SDSSJ0054-0946A,SDSSJ0054-0946B,Optical Colors,Slit Optical Spectroscopy,2.127,spec,2.110,spec,0:54:8.47,-9:46:38.3,13.535292,-9.777306,0:54:8.04,-9:46:25.7,13.533500,-9.773806,J2000,Optical,SDSS,J2000,Optical,SDSS,17.90,SDSS g PSF ext-corr,asinh,20.70,SDSS g PSF ext-corr,asinh,14.1,1629.938187,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
2,Binary Quasar,SDSSJ0117+3153A,SDSSJ0117+3153B,Optical Colors,Slit Optical Spectroscopy,2.626,spec,2.640,spec,1:17:8.39,+31:53:38.7,19.284958,31.894083,1:17:7.52,+31:53:41.2,19.281333,31.894778,J2000,Optical,SDSS,J2000,Optical,SDSS,20.42,SDSS g PSF ext-corr,asinh,19.77,SDSS g PSF ext-corr,asinh,11.3,-1152.211522,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
3,Binary Quasar,SDSSJ0201+0032A,SDSSJ0201+0032B,Optical Colors,Slit Optical Spectroscopy,2.299,spec,2.293,spec,2:1:43.49,+0:32:22.7,30.431208,0.539639,2:1:42.25,+0:32:18.5,30.426042,0.538472,J2000,Optical,SDSS,J2000,Optical,SDSS,19.42,SDSS g PSF ext-corr,asinh,20.33,SDSS g PSF ext-corr,asinh,19.0,544.295666,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
4,Binary Quasar,SDSSJ0248+0009A,SDSSJ0248+0009B,Optical Colors,Slit Optical Spectroscopy,1.641,spec,1.640,spec,2:48:20.8,+0:9:56.7,42.086667,0.165750,2:48:21.26,+0:9:57.3,42.088583,0.165917,J2000,Optical,SDSS,J2000,Optical,SDSS,19.26,SDSS g PSF ext-corr,asinh,20.73,SDSS g PSF ext-corr,asinh,6.9,113.236125,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
5,Binary Quasar,SDSSJ0332-0722A,SDSSJ0332-0722B,Optical Colors,Slit Optical Spectroscopy,2.102,spec,2.112,spec,3:32:38.38,-7:22:15.9,53.159917,-7.371083,3:32:37.19,-7:22:19.6,53.154958,-7.372111,J2000,Optical,SDSS,J2000,Optical,SDSS,20.35,SDSS g PSF ext-corr,asinh,20.63,SDSS g PSF ext-corr,asinh,18.1,-962.340604,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
6,Binary Quasar,SDSSJ0846+2749A,SDSSJ0846+2749B,Optical Colors,Slit Optical Spectroscopy,2.119,spec,2.114,spec,8:46:31.77,+27:49:21.9,131.632375,27.822750,8:46:30.38,+27:49:18.1,131.626583,27.821694,J2000,Optical,SDSS,J2000,Optical,SDSS,19.69,SDSS g PSF ext-corr,asinh,19.87,SDSS g PSF ext-corr,asinh,18.8,479.704488,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
7,Binary Quasar,SDSSJ0939+5953A,SDSSJ0939+5953B,Optical Colors,Slit Optical Spectroscopy,2.527,spec,2.530,spec,9:39:48.78,+59:53:48.7,144.953250,59.896861,9:39:46.56,+59:53:20.7,144.944000,59.889083,J2000,Optical,SDSS,J2000,Optical,SDSS,19.86,SDSS g PSF ext-corr,asinh,18.68,SDSS g PSF ext-corr,asinh,32.6,-254.215626,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
8,Binary Quasar,SDSSJ0955+6045A,SDSSJ0955+6045B,Optical Colors,Slit Optical Spectroscopy,0.716,spec,0.719,spec,9:55:24.37,+60:45:51.0,148.851542,60.764167,9:55:25.37,+60:45:33.8,148.855708,60.759389,J2000,Optical,SDSS,J2000,Optical,SDSS,20.37,SDSS g PSF ext-corr,asinh,20.65,SDSS g PSF ext-corr,asinh,18.6,-522.270344,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235
9,Binary Quasar,SDSSJ0959+5449A,SDSSJ0959+5449B,Optical Colors,Slit Optical Spectroscopy,1.954,spec,1.956,spec,9:59:7.46,+54:49:6.7,149.781083,54.818528,9:59:7.05,+54:49:8.4,149.779375,54.819000,J2000,Optical,SDSS,J2000,Optical,SDSS,20.07,

In [14]:
hennawi2006.to_csv('Hennawi2006_formattedwmagn.csv', sep=',')


In [15]:
#the_whills = hennawi2006

In [16]:
# Here we're loading in the Hennawi+2010 tables:

# Table 2 are binary quasars discovered from follow-up observations (sep 3 arcsec < delta theta < 60 arcsec)
hennawi2010 = (Table.read('Tables/Hennawi2010/table3.dat', readme = 'Tables/Hennawi2010/ReadMe', format='ascii.cds')).to_pandas()

primary_df = hennawi2010.iloc[::2].reset_index(drop=True)
secondary_df = hennawi2010.iloc[1::2].reset_index(drop=True)

# Append a suffix to the column names of primary and secondary DataFrames
primary_df.columns = [f"{col}_1" for col in primary_df.columns]
secondary_df.columns = [f"{col}_2" for col in secondary_df.columns]

hennawi2010 = pd.concat([primary_df, secondary_df], axis=1).reset_index(drop=True)

hennawi2010['Name1'] = hennawi2010['[HMS2010]_1']
hennawi2010['Name2'] = hennawi2010['[HMS2010]_2']

# Now creating appropriate RA and Dec coordinate columns:
hennawi2010['RA1'] = hennawi2010['RAh_1'].astype(str) + ':' + hennawi2010['RAm_1'].astype(str) + ':' + hennawi2010['RAs_1'].astype(str)
hennawi2010['Dec1'] = hennawi2010['DE-_1'].astype(str) + hennawi2010['DEd_1'].astype(str) + ':' + hennawi2010['DEm_1'].astype(str) + ':' + hennawi2010['DEs_2'].astype(str)

hennawi2010['RA2'] = hennawi2010['RAh_2'].astype(str) + ':' + hennawi2010['RAm_2'].astype(str) + ':' + hennawi2010['RAs_2'].astype(str)
hennawi2010['Dec2'] = hennawi2010['DE-_2'].astype(str) + hennawi2010['DEd_2'].astype(str) + ':' + hennawi2010['DEm_2'].astype(str) + ':' + hennawi2010['DEs_2'].astype(str)

# Now dropping the original 14 columns used for RA and Dec...
hennawi2010.drop(labels=['RAh_1','RAm_1','RAs_1','DE-_1','DEd_1','DEm_1','DEs_1', 'RAh_2','RAm_2','RAs_2','DE-_2','DEd_2','DEm_2','DEs_2','f_[HMS2010]_1','f_[HMS2010]_2'], axis=1, inplace=True)
hennawi2010.rename(columns={'z_1': 'z1', 'z_2': 'z2'}, inplace=True)
#hennawi2010.columns = hennawi2010.columns.str.replace('_', '')
#
## Dropping irrelevant magnitude columns (we likely won't be using these anyhow...)
#hennawi2010.drop(labels=['umag1','urmag1','rmag1','imag1','zmag1','umag2','rmag2','urmag2','imag2','zmag2'], axis=1, inplace=True)
#
# Adding informative columns about the coordinates
hennawi2010['Equinox1'] = "J2000"
hennawi2010['Coordinate_waveband1'] = "Optical"
hennawi2010['Coordinate_Source1'] = "SDSS"

hennawi2010['Equinox2'] = "J2000"
hennawi2010['Coordinate_waveband2'] = "Optical"
hennawi2010['Coordinate_Source2'] = "SDSS"

hennawi2010['System Type'] = 'Binary Quasar'
hennawi2010['z1_type'] = 'spec'
hennawi2010['z2_type'] = 'spec'
#
#hennawi2010['Brightness_band1'] = 'SDSS g PSF ext-corr'
#hennawi2010['Brightness_band2'] = 'SDSS g PSF ext-corr'
#
#hennawi2010['Brightness_type1'] = 'asinh'
#hennawi2010['Brightness_type2'] = 'asinh'
#
## Calculating here redshift difference (delta_z) and velocity differences dV
## dV is defined as: dv = (2.99e+5)*((1+z1)**2 - (1+z2)**2)/((1+z1)**2+(1+z2)**2)
## These come from  the work in Tytler+2009 for velocity differences and LOS distances derived from velocity differences
#
##v=H(z)*d, [H(z)/H0]**2 = ((1+z)**3)*Om0 + OmV
##    Therefore: H(z) = H0*np.sqrt(((1+z)**3)*Om0 + OmV)
##d = v/H(z)
#
##hennawi2010['delta_z'] = hennawi2010['z1'] - hennawi2010['z2']
hennawi2010['dV'] = (2.99e+5)*((1+hennawi2010['z1'])**2 - (1+hennawi2010['z2'])**2)/((1+hennawi2010['z1'])**2+(1+hennawi2010['z2'])**2)
## Defining it as dV_rwp for now since Hennawi+ also calculated this, and the values are discrepant with mine.
#
## The following will also calculate the LOS distance between the two objects
##hennawi2010['LOS_dist(Mpc)'] = np.abs(hennawi2010['dV_new'])/(70*np.sqrt(((1+hennawi2010['z1'])**3)*0.3 + 0.7))
#
## Now calculating the projected separation in kpc from the arcsec separation...
##hennawi2010['r_kpc'] = hennawi2010['r']/(0.72) # This was a check on Hennawi's calculations...
##hennawi2010['Sep(kpc)'] = hennawi2010['theta']*((cosmo.arcsec_per_kpc_proper(hennawi2010['z1']))**(-1))
## The separation is based upon the redshift of the first quasar in each pair, but the difference should be \
## minimal if z2 was used instead.
#

hennawi2010['Sep'] = hennawi2010['DelT_1']
## Now adding in a few columns to denote the confirmation method, author name+year, bibcode, and doi link
hennawi2010['Selection Method'] = ' '
hennawi2010['Confirmation Method'] = "Optical Spectroscopy"
hennawi2010['Paper(s)'] = "Hennawi+2010"
##hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'Paper(s)'] = "Hennawi+2006 ; Ellison+2007"
hennawi2010['BibCode(s)'] = "2010ApJ...719.1672H"
##hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'BibCode(s)'] = "2006AJ....131....1H ; 2007MNRAS.378..801E "
hennawi2010['DOI(s)'] = "https://doi.org/10.1088/0004-637X/719/2/1672"
##hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'DOI(s)'] = "https://doi.org/10.1086/498235 ; https://doi.org/10.1111/j.1365-2966.2007.11847.x"

##hennawi2010['PairID'] = hennawi2010.index
#
## Now removing all systems which do not meet our cut in velocity difference of ≤2000 km s**-1
##hennawi2010 = (hennawi2010[np.abs(hennawi2010['dV_rwp'])<=2000]).reset_index(drop=True)
## (This leaves 284 pairs)
#
### Here we're dropping the obvious duplicates where the A and B components are not mixed up between tables
##hennawi2010 = hennawi2010.drop_duplicates(subset=['Name1','Name2','z1','z2','theta','RA1','Dec1','RA2','Dec2'])
##hennawi2010 = hennawi2010.drop_duplicates(subset=['Name1','Name2']) #removing duplicates that have correct components only
##
### Here we need to do something clever, because Hennawi+2006 mixed up the A and B designations between different \
### tables I'm going to make two new columns: sum of z's and sum of gmag's
##hennawi2010['summedz'] = hennawi2010['z1']+hennawi2010['z2']
##hennawi2010['summedgmag'] = hennawi2010['gmag1']+hennawi2010['gmag2']
##
### Now this allows me to circumvent the issue of mixed up redshift and gmag columns. Nothing I can do about the \
### mixed up names, but NONE of the rows should have duplicate summed redshifts AND gmags AND identical separations \
### unless the rows are in fact the same pair.
##hennawi2010 = hennawi2010.drop_duplicates(subset=['summedz','summedgmag','theta'])
##
### Now dropping any remaining irrelevant columns:
##hennawi2010.drop(labels=['Sample','dV','chi2','PairID','summedz','summedgmag'], axis=1, inplace=True)
#
coordconvert = SkyCoord(ra = hennawi2010['RA1'], dec = hennawi2010['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2010['RA1_deg'] = coordconvert.ra.degree
hennawi2010['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = hennawi2010['RA2'], dec = hennawi2010['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2010['RA2_deg'] = coordconvert.ra.degree
hennawi2010['Dec2_deg'] = coordconvert.dec.degree

#hennawi2010.reset_index(drop=True, inplace=True)

hennawi2010

,[HMS2010]_1,umag_1,gmag_1,u_rmag_1,rmag_1,imag_1,zmag_1,chi2s_1,chi2p_1,z1,e_z_1,DelV_1,DelT_1,RT_1,[HMS2010]_2,umag_2,gmag_2,u_rmag_2,rmag_2,imag_2,zmag_2,chi2s_2,chi2p_2,z2,e_z_2,DelV_2,DelT_2,RT_2,Name1,Name2,RA1,Dec1,RA2,Dec2,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,z1_type,z2_type,dV,Sep,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),RA1_deg,Dec1_deg,RA2_deg,Dec2_deg
0,SDSS J0004-0844A,21.89,20.75,NaN,20.56,20.57,20.67,1.2,0.9,2.998,1450,650,4.4,35,SDSS J0004-0844B,22.80,21.02,NaN,20.87,20.84,20.66,4.6,0.3,2.989,1450,<NA>,NaN,<NA>,SDSS J0004-0844A,SDSS J0004-0844B,0:4:50.674,-8:44:51.97,0:4:50.92,-8:44:51.97,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,673.844143,4.4,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,1.211142,-8.747769,1.212167,-8.747769
1,SDSS J0829+2927A,23.64,20.93,NaN,20.28,19.68,19.30,63.3,22.5,3.054,1450,270,40.3,322,SDSS J0829+2927B,22.04,20.07,NaN,19.95,19.83,19.88,36.8,1.2,3.050,710,<NA>,NaN,<NA>,SDSS J0829+2927A,SDSS J0829+2927B,8:29:7.722,+29:27:18.1,8:29:9.018,+29:27:18.1,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,295.162811,40.3,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,127.282175,29.455028,127.287575,29.455028
2,SDSS J0956+2643A,21.61,19.43,NaN,19.22,19.28,19.30,143.2,2.6,3.087,520,150,16.5,131,SDSS J0956+2643B,22.71,20.78,NaN,20.46,20.46,20.32,14.1,0.6,3.085,520,<NA>,NaN,<NA>,SDSS J0956+2643A,SDSS J0956+2643B,9:56:27.149,+26:43:21.62,9:56:25.934,+26:43:21.62,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,146.353393,16.5,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,149.113121,26.722672,149.108058,26.722672
3,SDSS J0959+1032A,23.16,20.92,NaN,19.23,19.12,19.19,215.2,3.5,4.024,1450,250,44.1,320,SDSS J0959+1033B,25.86,21.47,NaN,20.00,19.56,19.50,24.5,1.7,4.020,1500,<NA>,NaN,<NA>,SDSS J0959+1032A,SDSS J0959+1033B,9:59:3.843,+10:32:25.02,9:59:5.139,+10:33:25.02,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,238.152093,44.1,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,149.766012,10.540283,149.771412,10.556950
4,SDSS J1016+4040A,21.26,19.42,NaN,19.22,19.10,18.84,109.8,8.1,2.996,520,1030,68.2,548,SDSS J1016+4040B,22.12,20.36,NaN,20.06,20.10,19.92,49.7,1.8,2.983,1450,<NA>,NaN,<NA>,SDSS J1016+4040A,SDSS J1016+4040B,10:16:1.509,+40:40:5.8,10:16:5.842,+40:40:5.8,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,974.304971,68.2,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,154.006288,40.668278,154.024342,40.668278
5,SDSS J1021+1112A,25.23,21.99,NaN,20.50,20.35,20.18,26.7,0.3,3.805,520,1470,7.6,56,SDSS J1021+1112B,25.46,22.36,NaN,20.62,20.73,20.26,58.2,5.4,3.829,630,<NA>,NaN,<NA>,SDSS J1021+1112A,SDSS J1021+1112B,10:21:16.982,+11:12:27.83,10:21:16.468,+11:12:27.83,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,-1489.714649,7.6,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,155.320758,11.207731,155.318617,11.207731
6,SDSS J1053+5001A,23.77,21.05,NaN,20.72,20.72,20.64,18.2,0.4,3.078,630,490,2.1,17,SDSS J1053+5001B,23.35,21.10,NaN,20.84,20.88,21.02,19.4,1.5,3.085,520,<NA>,NaN,<NA>,SDSS J1053+5001A,SDSS J1053+5001B,10:53:20.15,+50:1:47.84,10:53:20.041,+50:1:47.84,J2000,Optical,SDSS,J2000,Optical,SDSS,Binary Quasar,spec,spec,-512.801289,2.1,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,163.333958,50.029956,163.333504,50.029956
7,SDSS J1054+0215A,23.03,20.22,NaN,18.69,18.61,18.75,244.9,6.2,3.984,630,1120,88.3,644,SDSS J1054+0216B,23.21,20.58,NaN,19.41,19.10,18.97,12.9,5.3,3.966,790,<NA>,NaN,<NA>,SDSS J1054+0215A,SDSS J1054+0216B,10:54:34.174,+2:15:16.37,10:54:28.51

In [17]:
#the_whills

In [18]:
#from astropy.cosmology import FlatLambdaCDM 
#from astropy.cosmology import LambdaCDM #Creating our choice of cosmology here...
#cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

#value = (30)*(cosmo.arcsec_per_kpc_proper(1.5))**(-1)

In [19]:
#(30)*(cosmo.arcsec_per_kpc_proper(1.5))**(-1)

In [20]:
# Here we're loading in the catalog(s) from Myers+2007:

myers2007 = ((Table.read('Tables/Myers_2007/table1.dat', readme = 'Tables/Myers_2007/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['SDSSa','SDSSb']) 
# I've edited the ReadMe to remove the duplicate columns '---' which just contain the string 'SDSS' for each row
# They load in as SDSSa and SDSSb now, and they are immediately dropped from the dataframe (above). 

# (a) DR4CatalogArchive Server (e.g., likely Schneider et al. 2005). (b) Hennawi et al. (2006). \
# (c) Sramek & Weedman (1978) and Djorgovski & Spinrad (1984). (d ) Probable binary quasar-starburst \
# (Brotherton et al. 1999).

# Now combining the various coordinate columns to give us a RA and Dec column for each quasar in each pair
myers2007['RA1'] = myers2007['RAh'].astype(str) + ':' + myers2007['RAm'].astype(str) + ':' + myers2007['RAs'].astype(str)
myers2007['Dec1'] = myers2007['DE-'].astype(str) + myers2007['DEh'].astype(str) + ':' + myers2007['DEm'].astype(str) + ':' + myers2007['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = myers2007['RA1'], dec = myers2007['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

myers2007['RA1_deg'] = coordconvert.ra.degree
myers2007['Dec1_deg'] = coordconvert.dec.degree

# Now for the second set of coordinates
myers2007['RA2'] = myers2007['RA2h'].astype(str) + ':' + myers2007['RA2m'].astype(str) + ':' + myers2007['RA2s'].astype(str)
myers2007['Dec2'] = myers2007['DE2-'].astype(str) + myers2007['DE2h'].astype(str) + ':' + myers2007['DE2m'].astype(str) + ':' + myers2007['DE2s'].astype(str)

coordconvert = SkyCoord(ra = myers2007['RA2'], dec = myers2007['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

myers2007['RA2_deg'] = coordconvert.ra.degree
myers2007['Dec2_deg'] = coordconvert.dec.degree

# Adding in columns for information about the coordinates
myers2007['Equinox1'] = "J2000"
myers2007['Coordinate_waveband1'] = "Optical"
myers2007['Coordinate_Source1'] = "SDSS"

myers2007['Equinox2'] = "J2000"
myers2007['Coordinate_waveband2'] = "Optical"
myers2007['Coordinate_Source2'] = "SDSS"

myers2007['System Type'] = 'Binary Quasar'
myers2007['Selection Method'] = 'Optical Photometry'
myers2007['Confirmation Method'] = -100

myers2007['Brightness_band1'] = 'SDSS g'
myers2007['Brightness_type1'] = 'asinh mag'

myers2007['Brightness_band2'] = 'SDSS g'
myers2007['Brightness_type2'] = 'asinh mag'

# Now dropping the original 14 columns for separate RA and Dec components
myers2007.drop(labels=['RAh','RAm','RAs','DE-','DEh','DEm','DEs','RA2h','RA2m','RA2s','DE2-','DE2h','DE2m','DE2s'], axis=1, inplace=True)

# There is some overlap between Myers+2007 and other studies, like Hennawi+2006. 
# We will prioritize spectroscopic results over photometric results for our catalog. We see that some of the \
# overlap targets between Myers+ and other studies are noted as being quasar star pairs. Since these pairs \
# overlap with the spectroscopic study of Hennawi+ and others, we will remove these and assume these are genuinely \
# quasar star pairs, and therefore are not relevant for our study. 

# Removing the quasar-star pairs now...
myers2007 = (myers2007[(myers2007['n_zsp2']!="star b") & (myers2007['f_zsp1']!="star a")]).reset_index(drop=True)

# Brotheron+1999 list the redshift of both systems as being z=0.586, but Myers+07 notes only that System A has this
# spec-z. Here I am replacing the NaN values for zsp2 and N-zsp2...
myers2007.loc[myers2007.f_zsp1 == 'a,d', 'zsp2'] = 0.586
myers2007.loc[myers2007.f_zsp1 == 'a,d', 'n_zsp2'] = 'd'

# Making now a copy of this to compare to other studies and then removing all listings from other studies from the \
# main Myers+07 dataframe
myers2007_archive = myers2007.copy()

# Now we're going to split this dataframe in two: one with photometric pairs (which might have one spec-z) and\
# one where we have only spec-z pairs from other literature sources

myers2007_archive['zsp1'] = myers2007_archive['zsp1'].fillna(0)
myers2007_archive['zsp1b'] = myers2007_archive['zsp1b'].fillna(0)
myers2007_archive['zsp2'] = myers2007_archive['zsp2'].fillna(0)
myers2007_archive['zsp2b'] = myers2007_archive['zsp2b'].fillna(0)

# This one above has the spec-z pairs
myers2007_spec = myers2007_archive[((myers2007_archive['zsp1']>0) | (myers2007_archive['zsp1b']>0)) & ((myers2007_archive['zsp2']>0) | (myers2007_archive['zsp2b']>0))]
# This one below has the photometric pairs (where one may or may not have a spec-z)
myers2007_phot = (myers2007_archive.loc[myers2007_archive.index.difference(myers2007_spec.index)]).reset_index(drop=True)

myers2007_spec = (myers2007_spec).reset_index(drop=True) # Resetting the indices now...
myers2007_phot = (myers2007_phot).reset_index(drop=True)

# At this point, we essentially won't be using the _spec dataframe from Myers2007 as we will favor the data from \
# the spectroscopic studies instead.
# I manually checked the overlap between Myers+ and Hennawi+; the angular separations appear to agree between \
# the two paper, so I'll just stick with Hennawi's calculations

# Also note that the the columns zsp1b and zsp2b do not contain any additional redshift information, so we'll \
# drop the columns zsp1b, r_zsp1b, zsp2b, and r_zsp2b. We'll also drop the umag1 and umag2 columns
myers2007_phot.drop(labels=['zsp1b', 'r_zsp1b', 'zsp2b', 'r_zsp2b', 'umag1', 'umag2'], axis=1, inplace=True)

# Since we're not interested in the spectroscopic pairs from this table (they're included elsewhere, and no \
# new information is provided by Myers+07), I'm going to overwrite the dataframe myers2007 such that \
# myers2007 = myers2007_phot
myers2007 = myers2007_phot

# Now creating two columns: one for adopted redshift and one for adopted redshift type
# For sources with a spec-z, we'll adopt the spec-z. Otherwise we'll adopt the **MIDPOINT** phot-z of Myers+.

myers2007['z1'] = myers2007['zph1m'].where((myers2007['zsp1']==0), other=myers2007['zsp1'])
myers2007['z1_type'] = 'phot'
myers2007.loc[myers2007.zsp1>0, 'z1_type'] = 'spec'

myers2007['z2'] = myers2007['zph2m'].where((myers2007['zsp2']==0), other=myers2007['zsp2'])
myers2007['z2_type'] = 'phot'
myers2007.loc[myers2007.zsp2>0, 'z2_type'] = 'spec'

# Adding in information for the paper and catalog...
myers2007['Paper(s)'] = "Myers+2007"
myers2007['BibCode(s)'] = "2007ApJ...658...99M"
myers2007['DOI(s)'] = "https://doi.org/10.1086/511520"

# Keeping this here for now...
myers2007['PairID'] = myers2007.index



In [21]:
# 111 total pairs in Myers+2007
# --> 20 are spectroscopic pairs that overlap with other studies; removed, leaving 91 pairs.
# --> 3 instances of other studies finding that one of the 'quasars' was a star; removed, leaving 88 pairs

In [22]:
# Here we're loading in the catalog(s) from Myers+2008:

myers2008_t1 = ((Table.read('Tables/Myers2008/table1.dat', readme = 'Tables/Myers2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 1 contains DR4 KDE candidate binaries for which only one member has been observed to date

myers2008_t2 = ((Table.read('Tables/Myers2008/table2.dat', readme = 'Tables/Myers2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 2 contains confirmed binary quasars from DR4 KDE candidate sample

myers2008_t3 = (Table.read('Tables/Myers2008/table3.dat', readme = 'Tables/Myers2008/ReadMe', format='ascii.cds')).to_pandas()
# Table 3 contains the 10 projected pairs from the DR4 KDE candidate sample

myers2008_t4 = ((Table.read('Tables/Myers2008/table4.dat', readme = 'Tables/Myers2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 4 contains ambiguous pairs in the DR4 KDE binary quasar candidate sample

myers2008_t5 = (Table.read('Tables/Myers2008/table5.dat', readme = 'Tables/Myers2008/ReadMe', format='ascii.cds')).to_pandas()#.drop(columns=['---'])
# Table 5 contains previously identified DR4 KDE binary quasar candidates with 3''<theta<6'' separations.

# Now breaking this table into different tables to make removing sources easier...
myers2008_t5_pp = myers2008_t5[myers2008_t5['oc']=='PP'] # 12 projected pairs / non pairs
myers2008_t5_BQ = myers2008_t5[myers2008_t5['oc']=='BQ']
myers2008_t5_CL = myers2008_t5[myers2008_t5['oc']=='CL'] # 2 lenses

# Immediately we need to remove the quasars from Table 3 and the PP and CL quasars from Table 5 from the \
# Myers2007 catalog, which we'll do now...

# Note, the BQ's from Table 5 were already removed when we removed spectroscopic pairs that overlapped with \
# other catalogs/other papers.

In [23]:
# Removing the 10 projected pairs from Table 3 first...
myers_a = (myers2007[~myers2007['SDSS1'].isin(myers2008_t3['SDSS'])]).reset_index(drop=True)
myers_b = (myers_a[~myers_a['SDSS2'].isin(myers2008_t3['SDSS'])]).reset_index(drop=True)

# Now removing 12 projected pairs from Table 5. Some of these overlap with previously removed systems...
myers_c = (myers_b[~myers_b['SDSS1'].isin(myers2008_t5_pp['SDSS'])]).reset_index(drop=True)
myers_d = (myers_c[~myers_c['SDSS1'].isin(myers2008_t5_pp['SDSS'])]).reset_index(drop=True)
# This leaves us with 76 pairs...

# Now removing the 2 confirmed lenses from Table 5
# Based upon the coordinates listed in Myers+2008 Table 5, the sources have coordinates:
# 10 04 34.800 +41 12 39.29 --> SDSS1 = J100434.79+411239.2 (Verified manually)
# 12 06 29.648 +43 32 17.57 --> SDSS1 = J120629.65+433220.6 (verified manually)
lenses = pd.DataFrame({'SDSS1': ['J100434.79+411239.2','J120629.65+433220.6']})
myers_e = (myers_d[~myers_d['SDSS1'].isin(lenses['SDSS1'])]).reset_index(drop=True)
# This leaves us with 74 pairs


In [24]:
# Now manually updating the myers table (by matching their new nomenclature to the old by eye) for the \
# CONFIRMED binaries (spec-z's) from Table 2

# For J1158+1235A,B
# J115822.77+123518.5 --> 1158 A
# J115822.98+123520.3 --> 1158 B

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'zsp1'] = myers2008_t2['z'].iloc[0]
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'zsp2'] = myers2008_t2['z'].iloc[1]

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z1'] = myers2008_t2['z'].iloc[0]
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z2'] = myers2008_t2['z'].iloc[1]

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e['Notes'] = ''
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Notes'] = 'Spec-z from Myers+2008; J115822.77+123518.5: 1158 A and 115822.98+123520.3: 1158 B.'


# For J1320+3056A,B
# J132022.54+305622.8 --> J1320+3056A (gmag=18.60)
# J132022.64+305618.2 --> J1320+3056B (gmag=19.92)

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'zsp1'] = myers2008_t2['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'zsp2'] = myers2008_t2['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z1'] = myers2008_t2['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z2'] = myers2008_t2['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Notes'] = 'Spec-z from Myers+2008; J132022.54+305622.8: 1320 A and J132022.64+305618.2: 1320 B.'


# For J1418+2441A,B
# J141855.41+244108.9 --> J1418+2441A (gmag=19.27)
# J141855.53+244104.7 --> J1418+2441B (gmag=20.22)

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'zsp1'] = myers2008_t2['z'].iloc[4]
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'zsp2'] = myers2008_t2['z'].iloc[5]

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z1'] = myers2008_t2['z'].iloc[4]
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z2'] = myers2008_t2['z'].iloc[5]

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Notes'] = 'Spec-z from Myers+2008; J141855.41+244108.9: 1418 A and 141855.53+244104.7: 1418 B.'


# For J1426+0719A,B
# J142604.32+071930.0 --> J1426+0719B (gmag=20.12)
# J142604.26+071925.8 --> J1426+0719A (gmag=20.82)

# Bear in mind that we've had to flip the index order, as the A/B order flipped between catalogs
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'zsp1'] = myers2008_t2['z'].iloc[7]
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'zsp2'] = myers2008_t2['z'].iloc[6]

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z1'] = myers2008_t2['z'].iloc[7]
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z2'] = myers2008_t2['z'].iloc[6]

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Notes'] = 'Spec-z from Myers+2008; J142604.26+071925.8: 1426 A and J142604.32+071930.0: 1426 B.'


# For J1430+0714A,B
# J143002.88+071411.3 --> J1430+0714B (gmag=19.50)
# J143002.66+071415.6 --> J1430+0714A (gmag=20.27)

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'zsp1'] = myers2008_t2['z'].iloc[9]
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'zsp2'] = myers2008_t2['z'].iloc[8]

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z1'] = myers2008_t2['z'].iloc[9]
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z2'] = myers2008_t2['z'].iloc[8]

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Paper(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'DOI(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Notes'] = 'Spec-z from Myers+2008; J143002.66+071415.6: 1430 A and J143002.88+071411.3: 1430 B.'


# J1458+5448A,B
# J145826.72+544813.1 --> J1458+5448B (gmag=20.53)
# J145826.16+544814.8 --> J1458+5448A (gmag=20.79)

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'zsp1'] = myers2008_t2['z'].iloc[11]
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'zsp2'] = myers2008_t2['z'].iloc[10]

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z1'] = myers2008_t2['z'].iloc[11]
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z2'] = myers2008_t2['z'].iloc[10]

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Notes'] = 'Spec-z from Myers+2008; J145826.72+544813.1: 1458 B and J145826.16+544814.8: 1458 A.'


# J1606+2900A,B
# J160603.02+290050.8 --> J1606+2900B (gmag=18.42)
# J160602.81+290048.7 --> J1606+2900A (gmag=18.50)

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'zsp1'] = myers2008_t2['z'].iloc[13]
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'zsp2'] = myers2008_t2['z'].iloc[12]

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z1'] = myers2008_t2['z'].iloc[13]
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z2'] = myers2008_t2['z'].iloc[12]

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Notes'] = 'Spec-z from Myers+2008; J160603.02+290050.8: 1606 B and J160602.81+290048.7: 1606 A.'


# J1635+2911A,B
# J163510.30+291116.1 --> 1635+2911B (gmag=20.43)
# J163510.14+291120.6 --> 1635+2911A (gmag=18.83)

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'zsp1'] = myers2008_t2['z'].iloc[15]
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'zsp2'] = myers2008_t2['z'].iloc[14]

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z1'] = myers2008_t2['z'].iloc[15]
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z2'] = myers2008_t2['z'].iloc[14]

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Notes'] = 'Spec-z from Myers+2008; J163510.30+291116.1: 1635 B and J163510.14+291120.6: 1635 A.'



In [25]:
# Now adding in the spec-z's from the more ambiguous pairs in Table 4

# SDSS1=J093424.32+421130.8 and SDSS2=J093424.11+421135.0

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'zsp1'] = myers2008_t4['z'].iloc[1]
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'zsp2'] = myers2008_t4['z'].iloc[0] #This does not have a listed redshift

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z1'] = myers2008_t4['z'].iloc[1]
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z2'] = myers2008_t4['z'].iloc[0] # No listed redshift

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z1_type'] = 'spec'
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z2_type'] = 'spec' # No spec-z listed

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Notes'] = 'Spec-z from Myers+2008. Follow-up spectroscopy likely needed.'


# SDSS1=J120727.09+140817.1 and SDSS2=J120727.25+140820.3

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'zsp1'] = myers2008_t4['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'zsp2'] = myers2008_t4['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z1'] = myers2008_t4['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z2'] = myers2008_t4['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Notes'] = 'Spec-z from Myers+2008. Follow-up spectroscopy likely needed.'


# SDSS1=J123555.27+683627.0 --> J1235+6836B (gmag=)19.70
# SDSS2=J123554.78+683624.7 --> J1235+6836A (gmag=19.04)

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'zsp1'] = myers2008_t4['z'].iloc[5]
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'zsp2'] = myers2008_t4['z'].iloc[4]

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z1'] = myers2008_t4['z'].iloc[5]
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z2'] = myers2008_t4['z'].iloc[4]

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Notes'] = 'Spec-z from Myers+2008; J123555.27+683627.0: J1235 B and J123554.78+683624.7: J1235 A. Follow-up spectroscopy likely needed.'


# SDSS1=J150747.23+290333.2 --> J1507+2903B (gmag=19.97)
# SDSS2=J150746.90+290334.1 --> J1507+2903A (gmag=20.44)

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'zsp1'] = myers2008_t4['z'].iloc[7]
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'zsp2'] = myers2008_t4['z'].iloc[6]

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z1'] = myers2008_t4['z'].iloc[7]
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z2'] = myers2008_t4['z'].iloc[6]

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Notes'] = 'Spec-z from Myers+2008; J150747.23+290333.2: J1507 B and J150746.90+290334.1: J1507 A. Follow-up spectroscopy likely needed.'



In [26]:
# Here, we will break the myers master catalog into two: one for spec-z pairs, and the other for phot-z pairs or \
# phot-z/spec-z pairs

# This way, the spec pairs will be included in a uniform fashion with those in the master table, while a secondary \
# table for phot-z pairs will be included with the explicit mention that these require optical spectroscopic \
# follow-up

myers_master_spec = (myers_e[(myers_e['z1_type']=='spec') & (myers_e['z2_type']=='spec')]).reset_index(drop=True)
myers_master_phot = (myers_e[~myers_e['SDSS1'].isin(myers_master_spec['SDSS1'])]).reset_index(drop=True)

# And now we can remove the irrlevant photometric redshift columns and redundant spec-z columns from the \
# myers_master_spec table

myers_master_spec.drop(labels=['zph1l','zph1m','zph1u','zsp1','f_zsp1','zph2l','zph2m','zph2u','zsp2','n_zsp2',\
                              'PairID','f_Delta'], axis=1, inplace=True)
myers_master_spec['Confirmation Method'] = 'Optical Spectroscopy'

myers_master_phot.drop(labels=['PairID'], axis=1, inplace=True)
myers_master_phot['Notes'] = 'Spectroscopic follow-up is required.'
myers_master_phot['System Type'] = 'Binary Quasar Candidate'

In [27]:
# And we need to update the phot table with any remaining spec-z's that Myers found (see Table 1)
# There are 8 systems in which they have spec-z measurements for 1 out of 2 components in each pair

# SDSS1= J093015.01+420033.6  (gmag=20.03)
# SDSS2= J093014.81+420038.7 --> Has spec-z in Table 1 (gmag=19.71)

myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'zsp2'] = myers2008_t1['z'].iloc[0]
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'z2'] = myers2008_t1['z'].iloc[0]
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'z2_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J093521.02+641219.8 --> Has spec-z in Table 1 (gmag=20.99)
# SDSS2=J093521.80+641221.9   (gmag=20.96)

myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'zsp1'] = myers2008_t1['z'].iloc[2]
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'z1'] = myers2008_t1['z'].iloc[2]
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J095840.74+332216.3 --> Has spec-z in Table 1 (gmag=19.18)
# SDSS2=J095840.94+332211.5   (gmag=20.64)

myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'zsp1'] = myers2008_t1['z'].iloc[4]
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'z1'] = myers2008_t1['z'].iloc[4]
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J103939.31+100253.0 --> Has spec-z in Table 1 (gmag=18.42)
# SDSS2=J103939.53+100254.3  (gmag=19.60)

myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'zsp1'] = myers2008_t1['z'].iloc[6]
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'z1'] = myers2008_t1['z'].iloc[6]
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J162847.75+413045.4  (gmag=19.81)
# SDSS2=J162848.06+413043.1 --> Has a spec-z in Table 1 (gmag=20.40)

myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'zsp2'] = myers2008_t1['z'].iloc[9]
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'z2'] = myers2008_t1['z'].iloc[9]
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'z2_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'



In [28]:
# Now constructing the remaining columns for delta_z and dV for the spec pairs...
#myers_master_spec['delta_z'] = myers_master_spec['z1']-myers_master_spec['z2']
myers_master_spec['dV'] = (2.99e+5)*((1+myers_master_spec['z1'])**2 - (1+myers_master_spec['z2'])**2)/((1+myers_master_spec['z1'])**2+(1+myers_master_spec['z2'])**2)
#myers_master_spec['Sep(kpc)'] = myers_master_spec['Delta']*((cosmo.arcsec_per_kpc_proper(myers_master_spec['z1']))**(-1))

# Now renaming / rearranging columns for these two tables (to make concatenation easier later on...)

# First the spec table
myers_master_spec = myers_master_spec[['System Type','SDSS1','SDSS2','Selection Method','Confirmation Method',\
                                       'z1','z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2',\
                                       'Dec2','RA2_deg','Dec2_deg','Equinox1','Coordinate_waveband1',\
                                       'Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','gmag1','Brightness_band1','Brightness_type1','gmag2',\
                                       'Brightness_band2','Brightness_type2','Delta','dV',\
                                       'Paper(s)','BibCode(s)','DOI(s)','Notes']]

myers_master_spec.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method',\
                           'z1','z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2',\
                           'Dec2','RA2_deg','Dec2_deg','Equinox1','Coordinate_waveband1',\
                           'Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','Brightness1','Brightness_band1','Brightness_type1','Brightness2',\
                           'Brightness_band2','Brightness_type2','Sep','dV',\
                           'Paper(s)','BibCode(s)','DOI(s)','Notes']

# Now the phot table
myers_master_phot = myers_master_phot[['System Type','SDSS1','SDSS2','Selection Method','Confirmation Method',\
                                       'zph1l','zph1m','zph1u','zsp1','f_zsp1','z1','z1_type','zph2l','zph2m',\
                                       'zph2u','zsp2','n_zsp2','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg',\
                                       'RA2','Dec2','RA2_deg', 'Dec2_deg','Equinox1','Coordinate_waveband1',\
                                       'Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','gmag1','Brightness_band1','Brightness_type1','gmag2',\
                                       'Brightness_band2','Brightness_type2','Delta','f_Delta','Paper(s)',\
                                       'BibCode(s)','DOI(s)','Notes']]

myers_master_phot.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method',\
                           'zph1l','zph1m','zph1u','zsp1','f_zsp1','z1','z1_type','zph2l','zph2m',\
                           'zph2u','zsp2','n_zsp2','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg',\
                           'RA2','Dec2','RA2_deg','Dec2_deg','Equinox1','Coordinate_waveband1',\
                           'Coordinate_Source1','Equinox2','Coordinate_waveband2','Coordinate_Source2','Brightness1','Brightness_band1','Brightness_type1','Brightness2',\
                           'Brightness_band2','Brightness_type2','Sep','f_Sep','Paper(s)',\
                           'BibCode(s)','DOI(s)','Notes']
        

In [29]:
#myers_master_phot.columns

In [30]:
#myers_master_spec

In [31]:
#myers_master_phot

In [32]:
myers_master_table = pd.concat([myers_master_spec,myers_master_phot]).reset_index(drop=True)

myers_master_table


,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Paper(s),BibCode(s),DOI(s),Notes,zph1l,zph1m,zph1u,zsp1,f_zsp1,zph2l,zph2m,zph2u,zsp2,n_zsp2,f_Sep
0,Binary Quasar,J115822.77+123518.5,J115822.98+123520.3,Optical Photometry,Optical Spectroscopy,0.596,spec,0.596,spec,11:58:22.78,+12:35:18.6,179.594917,12.588500,11:58:22.99,+12:35:20.3,179.595792,12.588972,J2000,Optical,SDSS,J2000,Optical,SDSS,19.90,SDSS g,asinh mag,20.12,SDSS g,asinh mag,3.56,0.000000,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J115822.77+123518.5: 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Binary Quasar,J120727.09+140817.1,J120727.25+140820.3,Optical Photometry,Optical Spectroscopy,1.801,spec,1.800,spec,12:7:27.1,+14:8:17.2,181.862917,14.138111,12:7:27.26,+14:8:20.4,181.863583,14.139000,J2000,Optical,SDSS,J2000,Optical,SDSS,20.39,SDSS g,asinh mag,20.27,SDSS g,asinh mag,3.95,106.766645,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008. Follow-up spectroscopy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Binary Quasar,J123555.27+683627.0,J123554.78+683624.7,Optical Photometry,Optical Spectroscopy,1.514,spec,1.529,spec,12:35:55.27,+68:36:27.1,188.980292,68.607528,12:35:54.78,+68:36:24.8,188.978250,68.606889,J2000,Optical,SDSS,J2000,Optical,SDSS,19.70,SDSS g,asinh mag,19.04,SDSS g,asinh mag,3.51,-1778.687417,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J123555.27+683627.0: J...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Binary Quasar,J132022.54+305622.8,J132022.64+305618.2,Optical Photometry,Optical Spectroscopy,1.595,spec,1.596,spec,13:20:22.55,+30:56:22.9,200.093958,30.939694,13:20:22.64,+30:56:18.3,200.094333,30.938417,J2000,Optical,SDSS,J2000,Optical,SDSS,18.60,SDSS g,asinh mag,19.92,SDSS g,asinh mag,4.74,-115.199379,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J132022.54+305622.8: 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Binary Quasar,J141855.41+244108.9,J141855.53+244104.7,Optical Photometry,Optical Spectroscopy,0.572,spec,0.573,spec,14:18:55.42,+24:41:8.9,214.730917,24.685806,14:18:55.54,+24:41:4.7,214.731417,24.684639,J2000,Optical,SDSS,J2000,Optical,SDSS,19.27,SDSS g,asinh mag,20.22,SDSS g,asinh mag,4.50,-190.143065,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J141855.41+244108.9: 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Binary Quasar,J142604.32+071930.0,J142604.26+071925.8,Optical Photometry,Optical Spectroscopy,1.309,spec,1.312,spec,14:26:4.33,+7:19:30.0,216.518042,7.325000,14:26:4.27,+7:19:25.9,216.517792,7.323861,J2000,Optical,SDSS,J2000,Optical,SDSS,20.12,SDSS g,asinh mag,20.82,SDSS g,asinh mag,4.27,-388.227493,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J142604.26+071925.8: 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Binary Quasar,J143002.88+071411.3,J143002.66+071415.6,Optical Photometry,Optical Spectroscopy,1.261,spec,1.246,spec,14:30:2.89,+7:14:11.3,217.512042,7.236472,14:30:2.66,+7:14:15.6,217.511083,7.237667,J2000,Optical,SDSS,J2000,Optical,SDSS,19.50,SDSS g,asinh mag,20.27,SDSS g,asinh mag,5.41,1990.215364,Myers+2007 ; Myers+2008,2007ApJ...658...99M ; 2008ApJ...678..635M,https://doi.org/10.1086/511520 ; https://doi.o...,Spec-z from Myers+2008; J143002.66+071415.6: 1...,N

In [33]:
# Here we will match together the Hennawi+ and Myers+ tables. 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(hennawi2006,myers_master_table,10)

print(len(tmatches_1),len(tmatches_2))

the_whills = pd.concat([hennawi2006,myers_master_table])
# verified that are currently no matches between the hennawi+2006 and myers+ table as they are currently designed
# *****
# However, I previously stripped out the matching pairs from the myers+ table; I need to back, identify the \
# matches, and add in the bib information for those targets.
# *****


0 0


In [34]:
# Here we will match together the Hennawi+ and Myers+ tables. 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(the_whills,hennawi2010,20)

print(len(tmatches_1),len(tmatches_2))

# Just a note: when you match against the individual targets, you will match to J1439-003 which was identified \
# by Schneider+2000

# there are two matches; one match whne comparing RA1 and RA1, and 2 matches comparing RA1 and RA2
for index, row in the_whills.iterrows():
    if index in idx1_2:
        the_whills.at[index, 'Paper(s)'] += ' ; Hennawi+2010'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2010ApJ...719.1672H' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/719/2/1672'

hennawi2010.drop(idx2_2,inplace=True)
# dropping the 2 matches

the_whills = pd.concat([the_whills,hennawi2010])

# verified that the matching process works properly



2 4


In [35]:
the_whills

,level_0,index,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Paper(s),BibCode(s),DOI(s),Table_flag,Notes,zph1l,zph1m,zph1u,zsp1,f_zsp1,zph2l,zph2m,zph2u,zsp2,n_zsp2,f_Sep,[HMS2010]_1,umag_1,gmag_1,u_rmag_1,rmag_1,imag_1,zmag_1,chi2s_1,chi2p_1,e_z_1,DelV_1,DelT_1,RT_1,[HMS2010]_2,umag_2,gmag_2,u_rmag_2,rmag_2,imag_2,zmag_2,chi2s_2,chi2p_2,e_z_2,DelV_2,DelT_2,RT_2
0,0.0,0,Binary Quasar,SDSSJ0048-1051A,SDSSJ0048-1051B,Optical Colors,Slit Optical Spectroscopy,1.555,spec,1.560,spec,0:48:0.77,-10:51:48.6,12.003208,-10.863500,0:48:0.96,-10:51:46.2,12.004000,-10.862833,J2000,Optical,SDSS,J2000,Optical,SDSS,20.52,SDSS g PSF ext-corr,asinh,20.05,SDSS g PSF ext-corr,asinh,3.60,-584.554671,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
1,1.0,1,Binary Quasar,SDSSJ0054-0946A,SDSSJ0054-0946B,Optical Colors,Slit Optical Spectroscopy,2.127,spec,2.110,spec,0:54:8.47,-9:46:38.3,13.535292,-9.777306,0:54:8.04,-9:46:25.7,13.533500,-9.773806,J2000,Optical,SDSS,J2000,Optical,SDSS,17.90,SDSS g PSF ext-corr,asinh,20.70,SDSS g PSF ext-corr,asinh,14.10,1629.938187,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
2,2.0,2,Binary Quasar,SDSSJ0117+3153A,SDSSJ0117+3153B,Optical Colors,Slit Optical Spectroscopy,2.626,spec,2.640,spec,1:17:8.39,+31:53:38.7,19.284958,31.894083,1:17:7.52,+31:53:41.2,19.281333,31.894778,J2000,Optical,SDSS,J2000,Optical,SDSS,20.42,SDSS g PSF ext-corr,asinh,19.77,SDSS g PSF ext-corr,asinh,11.30,-1152.211522,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
3,3.0,3,Binary Quasar,SDSSJ0201+0032A,SDSSJ0201+0032B,Optical Colors,Slit Optical Spectroscopy,2.299,spec,2.293,spec,2:1:43.49,+0:32:22.7,30.431208,0.539639,2:1:42.25,+0:32:18.5,30.426042,0.538472,J2000,Optical,SDSS,J2000,Optical,SDSS,19.42,SDSS g PSF ext-corr,asinh,20.33,SDSS g PSF ext-corr,asinh,19.00,544.295666,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
4,4.0,4,Binary Quasar,SDSSJ0248+0009A,SDSSJ0248+0009B,Optical Colors,Slit Optical Spectroscopy,1.641,spec,1.640,spec,2:48:20.8,+0:9:56.7,42.086667,0.165750,2:48:21.26,+0:9:57.3,42.088583,0.165917,J2000,Optical,SDSS,J2000,Optical,SDSS,19.26,SDSS g PSF ext-corr,asinh,20.73,SDSS g PSF ext-corr,asinh,6.90,113.236125,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
5,5.0,5,Binary Quasar,SDSSJ0332-0722A,SDSSJ0332-0722B,Optical Colors,Slit Optical Spectroscopy,2.102,spec,2.112,spec,3:32:38.38,-7:22:15.9,53.159917,-7.371083,3:32:37.19,-7:22:19.6,53.154958,-7.372111,J2000,Optical,SDSS,J2000,Optical,SDSS,20.35,SDSS g PSF ext-corr,asinh,20.63,SDSS g PSF ext-corr,asinh,18.10,-962.340604,Hennawi+2006,2006AJ....131....1H,https://doi.org/10.1086/498235,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>
6,6.0,6,Binary Quasar,SDSSJ0846+2749A,SDSSJ0846+2749B,Optical Colors,Slit

In [36]:
# Adding in citations for Vignali+2018 here

vigobjs = ['SDSS J1622+0702A','SDSS J1307+0422A']

# Adding the DOI, author, and bibcode info 
for index, row in the_whills.iterrows():
    if (row['Name1'] in vigobjs) or (row['Name2'] in vigobjs):
        the_whills.at[index, 'Paper(s)'] += ' ; Vignali+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018MNRAS.477..780V' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty682'
        the_whills.at[index, 'Notes'] += ' The BAL QSO in this system is heavily obscured in the X-rays (>10^23)'

for i in the_whills['Paper(s)']:
    if 'Vignali+2018' in i:
        print('True')

# this appears to be verified





True
True
True
True


In [37]:
#the_whills

In [38]:
# Now loading in the catalog(s) from Kirkman+2008
kirkman2008 = (Table.read('Tables/Kirkman_2008/table1.dat', readme = 'Tables/Kirkman_2008/ReadMe', format='ascii.cds')).to_pandas()

# Creating RA and Dec columns for the foreground and background quasars
kirkman2008['RA1'] = kirkman2008['RAh'].astype(str) + ':' + kirkman2008['RAm'].astype(str) + ':' + kirkman2008['RAs'].astype(str)
kirkman2008['Dec1'] = kirkman2008['DE-'].astype(str) + kirkman2008['DEd'].astype(str) + ':' + kirkman2008['DEm'].astype(str) + ':' + kirkman2008['DEs'].astype(str)

kirkman2008['RA2'] = kirkman2008['RA2h'].astype(str) + ':' + kirkman2008['RA2m'].astype(str) + ':' + kirkman2008['RA2s'].astype(str)
kirkman2008['Dec2'] = kirkman2008['DE2-'].astype(str) + kirkman2008['DE2d'].astype(str) + ':' + kirkman2008['DE2m'].astype(str) + ':' + kirkman2008['DE2s'].astype(str)

# Now dropping the original 14 columns for separate RA and Dec components
kirkman2008.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs','RA2h','RA2m','RA2s','DE2-','DE2d','DE2m','DE2s'], axis=1, inplace=True)

# Dropping irrelevant columns, such as S/N columns 'S', 'S2', as well as the impact parameter 'b', the Lyman-alpha
# luminosity limit 'LLya', and the expected UV flux ratio 'OmeMax'
kirkman2008.drop(labels=['S', 'S2','b','LLya','OmeMax'], axis=1, inplace=True)

# Adding in columns for information about the coordinates
kirkman2008['Equinox1'] = "J2000"
kirkman2008['Coordinate_waveband1'] = "NA"
kirkman2008['Coordinate_Source1'] = "NA"

kirkman2008['Equinox2'] = "J2000"
kirkman2008['Coordinate_waveband2'] = "NA"
kirkman2008['Coordinate_Source2'] = "NA"

kirkman2008['Brightness_type1'] = '-100'
kirkman2008['Brightness_type2'] = '-100'

# Calculating the redshift difference and velocity difference between the quasar pairs
#kirkman2008['delta_z'] = kirkman2008['zem'] - kirkman2008['zem2']
kirkman2008['dV'] = (2.99e+5)*((1+kirkman2008['zem'])**2 - (1+kirkman2008['zem2'])**2)/((1+kirkman2008['zem'])**2+(1+kirkman2008['zem2'])**2)
#kirkman2008['Sep(kpc)'] = kirkman2008['Sep']*((cosmo.arcsec_per_kpc_proper(kirkman2008['zem']))**(-1))

# Now adding in columns for the author+year, bibcode, and doi link
kirkman2008['Paper(s)'] = "Kirkman+2008"
kirkman2008['BibCode(s)'] = "2008MNRAS.391.1457K"
kirkman2008['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.13994.x"

# Removing quasar pairs with velocity differences |v|>2000 km/s 
kirkman2008_master = (kirkman2008[np.abs(kirkman2008['dV'])<=2000]).reset_index(drop=True)

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = kirkman2008_master['RA1'], dec = kirkman2008_master['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

kirkman2008_master['RA1_deg'] = coordconvert.ra.degree
kirkman2008_master['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = kirkman2008_master['RA2'], dec = kirkman2008_master['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

kirkman2008_master['RA2_deg'] = coordconvert.ra.degree
kirkman2008_master['Dec2_deg'] = coordconvert.dec.degree


In [39]:
#kirkman2008_master

In [40]:
# Here we will match together the Kirkman+ and the overarching table 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(the_whills,kirkman2008_master,10)

print(len(tmatches_1),len(tmatches_2))

#tmatches_1
#tmatches_2

# Okay, there are 2 matches when examining RA1 and RA1 from Tables 1 and 2, 8 matches when matching RA1 to RA2
# in tables 1 and 2

# It seems that Kirkman+ simply selected objects already identified by Hennawi+ for their study. We are just \
# adding bib information 

for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Kirkman+2008'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2008MNRAS.391.1457K' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1111/j.1365-2966.2008.13994.x'

for index, row in the_whills.iterrows():
    if index in idx1_2:
        the_whills.at[index, 'Paper(s)'] += ' ; Kirkman+2008'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2008MNRAS.391.1457K' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1111/j.1365-2966.2008.13994.x'

#for i in the_whills['Paper(s)']:
#    if 'Kirkman+2008' in i:
#        print('True')
#

# verified now


4 16


In [41]:
#the_whills

In [42]:
# Now loading in the catalog(s) from Tytler+2009
tytler2009 = (Table.read('Tables/Tytler_2009/table1.dat', readme = 'Tables/Tytler_2009/ReadMe', format='ascii.cds')).to_pandas()

# Table 4 looks irrelevant for our purposes, so we're not loading that in.

# Creating an RA and Dec column for each source
tytler2009['RA'] = tytler2009['RAh'].astype(str) + ':' + tytler2009['RAm'].astype(str) + ':' + tytler2009['RAs'].astype(str)
tytler2009['Dec'] = tytler2009['DE-'].astype(str) + tytler2009['DEd'].astype(str) + ':' + tytler2009['DEm'].astype(str) + ':' + tytler2009['DEs'].astype(str)

# Dropping the original 7 columns for RA and Dec
tytler2009.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs'], axis=1, inplace=True)

# Now separating pairs from triples, and taking each list - where the pairs are listed across two rows each - and \
# realigning the information so that each pair inhabits one row. This will make calculations easier.

tytler2009['PairID'] = tytler2009['Pair']

# Grouping the objects by Pair ID and breaking the table into two adjust the formatting...

tytler2009_pairs = (tytler2009.groupby('Pair').filter(lambda x: len(x['PairID'])<=2)).reset_index(drop=True)
tytler2009_pairs.drop(labels=['PairID'], axis=1, inplace=True)

tytler2009_3s = (tytler2009.groupby('Pair').filter(lambda x: len(x['PairID'])>2)).reset_index(drop=True)
tytler2009_3s.drop(labels=['PairID'], axis=1, inplace=True)

# Now slicing the pairs table and then joining them
tytler_pairs1 = tytler2009_pairs[0::2] # Slicing now only the A components
tytler_pairs2 = tytler2009_pairs[1::2] # Slicing now only the B components

tytler2009_pairs = tytler_pairs1.join(tytler_pairs2.set_index('Pair'), on='Pair', how='left', lsuffix = '1', rsuffix='2')

# Now doing the same for the triples...
tytler2009_3s1 = tytler2009_3s[0::3] # Slicing now only the A components
tytler2009_3s2 = tytler2009_3s[1::3] # Slicing now only the B components
tytler2009_3s3 = tytler2009_3s[2::3] # Slicing now only the C components

# Renaming the columns for the triplets so that they are easier to work with
tytler2009_3s1.columns=['zem1','---1','Pair','m_Pair1','Sep12','RA1','Dec1']
tytler2009_3s2.columns=['zem2','---2','Pair','m_Pair2','Sep23','RA2','Dec2']
tytler2009_3s3.columns=['zem3','---3','Pair','m_Pair3','Sep13','RA3','Dec3']

# Combining the triplets together
tytler2009_3s = tytler2009_3s1.join(tytler2009_3s2.set_index('Pair'), on='Pair', how='left', lsuffix = '1', rsuffix='2')
tytler2009_3 = tytler2009_3s.join(tytler2009_3s3.set_index('Pair'), on='Pair', how='left', rsuffix='3')

# Adding in columns for information about the coordinates
tytler2009_pairs['Equinox1'] = "J2000"
tytler2009_pairs['Coordinate_waveband1'] = "NA"
tytler2009_pairs['Coordinate_Source1'] = "NA"

tytler2009_pairs['Equinox2'] = "J2000"
tytler2009_pairs['Coordinate_waveband2'] = "NA"
tytler2009_pairs['Coordinate_Source2'] = "NA"

tytler2009_3['Equinox1'] = "J2000"
tytler2009_3['Coordinate_waveband1'] = "NA"
tytler2009_3['Coordinate_Source1'] = "NA"

tytler2009_3['Equinox2'] = "J2000"
tytler2009_3['Coordinate_waveband2'] = "NA"
tytler2009_3['Coordinate_Source2'] = "NA"

tytler2009_3['Equinox3'] = "J2000"
tytler2009_3['Coordinate_waveband3'] = "NA"
tytler2009_3['Coordinate_Source3'] = "NA"

# Calculating the redshift difference and velocity difference between the quasar pairs
#tytler2009_pairs['delta_z'] = tytler2009_pairs['zem1'] - tytler2009_pairs['zem2']
tytler2009_pairs['dV'] = (2.99e+5)*((1+tytler2009_pairs['zem1'])**2 - (1+tytler2009_pairs['zem2'])**2)/((1+tytler2009_pairs['zem1'])**2+(1+tytler2009_pairs['zem2'])**2)
#tytler2009_pairs['Sep(kpc)'] = tytler2009_pairs['Sep1']*((cosmo.arcsec_per_kpc_proper(tytler2009_pairs['zem1']))**(-1))

#tytler2009_3['delta_z12'] = tytler2009_3['zem1'] - tytler2009_3['zem2']
tytler2009_3['dV12'] = (2.99e+5)*((1+tytler2009_3['zem1'])**2 - (1+tytler2009_3['zem2'])**2)/((1+tytler2009_3['zem1'])**2+(1+tytler2009_3['zem2'])**2)
#tytler2009_3['Sep(kpc)12'] = tytler2009_3['Sep12']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem1']))**(-1))

#tytler2009_3['delta_z23'] = tytler2009_3['zem2'] - tytler2009_3['zem3']
tytler2009_3['dV23'] = (2.99e+5)*((1+tytler2009_3['zem2'])**2 - (1+tytler2009_3['zem3'])**2)/((1+tytler2009_3['zem2'])**2+(1+tytler2009_3['zem3'])**2)
#tytler2009_3['Sep(kpc)23'] = tytler2009_3['Sep23']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem2']))**(-1))

#tytler2009_3['delta_z13'] = tytler2009_3['zem1'] - tytler2009_3['zem3']
tytler2009_3['dV13'] = (2.99e+5)*((1+tytler2009_3['zem1'])**2 - (1+tytler2009_3['zem3'])**2)/((1+tytler2009_3['zem1'])**2+(1+tytler2009_3['zem3'])**2)
#tytler2009_3['Sep(kpc)13'] = tytler2009_3['Sep13']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem1']))**(-1))

# Now adding in columns for the author+year, bibcode, and doi link
tytler2009_pairs['Paper(s)'] = "Tytler+2009"
tytler2009_pairs['BibCode(s)'] = "2009MNRAS.392.1539T"
tytler2009_pairs['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.14159.x"

tytler2009_3['Paper(s)'] = "Tytler+2009"
tytler2009_3['BibCode(s)'] = "2009MNRAS.392.1539T"
tytler2009_3['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.14159.x"

# Now applying a velocity difference cut of dV <= 2000 km s**-1 to ensure we are selecting systems which are \
# physically associated and not projected

tytler2009_pairs_master = (tytler2009_pairs[np.abs(tytler2009_pairs['dV'])<=2000]).reset_index(drop=True)

tytler2009_3_master = (tytler2009_3[(tytler2009_3['dV12']<=2000) & (tytler2009_3['dV13']<=2000) & (tytler2009_3['dV23']<=2000)]).reset_index(drop=True)

# Now dropping any remaining irrelevant columns: '---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'
tytler2009_pairs_master.drop(labels=['---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'], axis=1, inplace=True)

#tytler2009_3_master.drop(labels=['---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'], axis=1, inplace=True)

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = tytler2009_pairs_master['RA1'], dec = tytler2009_pairs_master['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_pairs_master['RA1_deg'] = coordconvert.ra.degree
tytler2009_pairs_master['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = tytler2009_pairs_master['RA2'], dec = tytler2009_pairs_master['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_pairs_master['RA2_deg'] = coordconvert.ra.degree
tytler2009_pairs_master['Dec2_deg'] = coordconvert.dec.degree

# Adding a column that stores the index value, which we'll use soon to remove duplicate sources that appear in \
# both the Kirkman and Tytler catalogs
tytler2009_pairs_master['index'] = tytler2009_pairs_master.index

# Update on 6 December 2023
# Adding in the separations, redshifts, and adding in system types and naming conventions
tytler2009_pairs_master['Sep'] = tytler2009_pairs_master['Sep1']
tytler2009_pairs_master['z1'] = tytler2009_pairs_master['zem1']
tytler2009_pairs_master['z2'] = tytler2009_pairs_master['zem2']

## Convert RA and Dec to the desired format
#coords = SkyCoord(ra=tytler2009_pairs_master['RA1']u.hour, dec=tytler2009_pairs_master['Dec1']*u.degree, frame='icrs')
#ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
#dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)
#
## Concatenate to form J_format using a loop
#tytler2009_pairs_master['J_format'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]
#
#tytler2009_pairs_master['Name1'] = tytler2009_pairs_master['J_format']#tytler2009_pairs_master['zem2']
##tytler2009_pairs_master['Name2'] = #tytler2009_pairs_master['zem2']
#
## here we're droppping the duplicate columns from yuan's tables:
##tytler2009_pairs_master.drop_duplicates(subset='J_format', keep='last', inplace=True)

tytler2009_pairs_master['System Type'] = 'Quasar Pair'


In [43]:
tytler2009_pairs_master


,zem1,Sep1,RA1,Dec1,zem2,RA2,Dec2,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,dV,Paper(s),BibCode(s),DOI(s),RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,index,Sep,z1,z2,System Type
0,1.878,88.64,0:44:34.08,+0:19:3.5,1.866,0:44:39.33,+0:18:22.8,J2000,NA,NA,J2000,NA,NA,1249.298169,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,11.142000,0.317639,11.163875,0.306333,0,88.64,1.878,1.866,Quasar Pair
1,1.970,12.04,2:9:54.8,-10:2:23.0,1.976,2:10:0.1,-10:3:54.0,J2000,NA,NA,J2000,NA,NA,-603.430263,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,32.478333,-10.039722,32.500417,-10.065000,1,12.04,1.970,1.976,Quasar Pair
2,2.554,72.04,3:10:36.47,-30:51:8.4,2.544,3:10:41.06,-30:50:27.5,J2000,NA,NA,J2000,NA,NA,842.489170,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,47.651958,-30.852333,47.671083,-30.840972,2,72.04,2.554,2.544,Quasar Pair
3,2.043,149.34,11:45:47.55,-0:31:6.7,2.055,11:45:53.67,-0:33:4.5,J2000,NA,NA,J2000,NA,NA,-1176.774715,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,176.448125,-0.518528,176.473625,-0.551250,3,149.34,2.043,2.055,Quasar Pair
4,2.473,74.01,12:12:51.14,-0:53:42.2,2.459,12:12:56.06,-0:53:36.5,J2000,NA,NA,J2000,NA,NA,1207.727330,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,183.213083,-0.895056,183.233583,-0.893472,4,74.01,2.473,2.459,Quasar Pair
5,2.193,299.36,14:20:45.98,-0:5:18.0,2.193,14:20:55.61,-0:9:40.0,J2000,NA,NA,J2000,NA,NA,0.000000,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,215.191583,-0.088333,215.231708,-0.161111,5,299.36,2.193,2.193,Quasar Pair
6,2.014,117.22,16:12:37.9,+23:57:9.0,2.005,16:12:45.6,+23:58:0.0,J2000,NA,NA,J2000,NA,NA,894.166467,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,243.157917,23.952500,243.190000,23.966667,6,117.22,2.014,2.005,Quasar Pair
7,2.127,244.99,17:30:14.71,+54:56:57.5,2.112,17:30:42.38,+54:56:1.1,J2000,NA,NA,J2000,NA,NA,1437.722095,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,262.561292,54.949306,262.676583,54.933639,7,244.99,2.127,2.112,Quasar Pair
8,1.930,49.52,23:9:11.88,-27:32:27.1,1.927,23:9:15.34,-27:32:45.3,J2000,NA,NA,J2000,NA,NA,306.300073,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,347.299500,-27.540861,347.313917,-27.545917,8,49.52,1.930,1.927,Quasar Pair
9,1.883,24.82,11:7:25.7,+0:33:53.6,1.882,11:7:27.08,+0:34:7.3,J2000,NA,NA,J2000,NA,NA,103.729398,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,166.857083,0.564889,166.862833,0.568694,9,24.82,1.883,1.882,Quasar Pair


In [44]:
# Here we will match together the Tytler+ pairs and the overarching table 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(the_whills,tytler2009_pairs_master,10)

print(len(tmatches_1),len(tmatches_2))

#tmatches_1
#tmatches_2

# Okay, there is 1 matches when examining RA1 and RA1 from Tables 1 and 2, 7 matches when matching RA1 to RA2
# in tables 1 and 2



for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Tytler+2009'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2009MNRAS.392.1539T' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1111/j.1365-2966.2008.13994.x'

for index, row in the_whills.iterrows():
    if index in idx1_2:
        the_whills.at[index, 'Paper(s)'] += ' ; Tytler+2009'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2009MNRAS.392.1539T' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1111/j.1365-2966.2008.14159.x'

#for i in the_whills['Paper(s)']:
#    if 'Tytler+2009' in i:
#        print('True')

tytler2009_pairs_master.drop(idx2,inplace=True)
tytler2009_pairs_master.drop(idx2_2,inplace=True)
# dropping the 2 matches

the_whills = pd.concat([the_whills,tytler2009_pairs_master])

# verified that this works properly


2 14


In [45]:
# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = tytler2009_3_master['RA1'], dec = tytler2009_3_master['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_3_master['RA1_deg'] = coordconvert.ra.degree
tytler2009_3_master['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = tytler2009_3_master['RA2'], dec = tytler2009_3_master['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_3_master['RA2_deg'] = coordconvert.ra.degree
tytler2009_3_master['Dec2_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = tytler2009_3_master['RA3'], dec = tytler2009_3_master['Dec3'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_3_master['RA3_deg'] = coordconvert.ra.degree
tytler2009_3_master['Dec3_deg'] = coordconvert.dec.degree


# Here we will match together the Tytler+ triples and the overarching table 
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2, tunique_3, tmatches_3, idx1_3, idx2_3 = match_tables_fib_triples(the_whills,tytler2009_3_master,15)

print(len(tmatches_1),len(tmatches_2),len(tmatches_3))
# Alright, there are apparently no matches between Object 1 in the_whills and the objects in slots 1, 2, or 3 \
# in the Tytler+ table.
# We'll keep the duals and triples separate for now and save the triple table here for later collection.

#tmatches_1
#tmatches_2

# verified


0 0 0


In [46]:
# saving the tylter+ triples table now...
tytler2009_3_master.to_csv('Tytler_triples_table_DR1.csv', sep=',', index=False)


In [47]:
# Now loading in Gattano+2014 here

gattano2014 = ((Table.read('Tables/Gattano2014/table7.dat', readme = 'Tables/Gattano2014/ReadMe', format='ascii.cds')).to_pandas()) 

#gattano2014['Name'] = gattano2014['QSO1']
#gattano2014['Name2'] = gattano2014['QSO2']
#gattano2014['z2'] = gattano2014['z2']
#gattano2014['z1_type'] = "spec"
#gattano2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(gattano2014,gattano2014['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = gattano2014['RA'], dec = gattano2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#gattano2014['RA1_deg'] = coordconvert.ra.degree
#gattano2014['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#gattano2014['RA2'] = gattano2014['RA']
#gattano2014['Dec2'] = gattano2014['Dec']

#gattano2014['RA2_deg'] = gattano2014['RA1_deg']
#gattano2014['Dec2_deg'] = gattano2014['Dec1_deg']

# Adding details about the coordinates
gattano2014['Equinox1'] = "J2000"
gattano2014['Coordinate_waveband1'] = "Optical"
gattano2014['Coordinate_Source1'] = "-99"

gattano2014['Equinox2'] = "J2000"
gattano2014['Coordinate_waveband2'] = "Optical"
gattano2014['Coordinate_Source2'] = "-99"

gattano2014['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
gattano2014['Brightness1'] = -100
gattano2014['Brightness_band1'] = -100
gattano2014['Brightness_type1'] = -100

gattano2014['Brightness2'] = -100
gattano2014['Brightness_band2'] = -100
gattano2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
gattano2014['Sep'] = gattano2014['theta'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#gattano2014['Sep(kpc)'] = gattano2014['Sep']*((cosmo.arcsec_per_kpc_proper(gattano2014['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#gattano2014['delta_z'] = gattano2014['z']-gattano2014['z2']
gattano2014['dV'] = (2.99e+5)*((1+gattano2014['z1'])**2 - (1+gattano2014['z2'])**2)/((1+gattano2014['z1'])**2+(1+gattano2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
gattano2014['Selection Method'] = "Complex. Needs review." #DPSELs
gattano2014['Confirmation Method'] = "-99"
gattano2014['Paper(s)'] = "Gattano+2014"
gattano2014['BibCode(s)'] = "2014A&A...564A.117G"
gattano2014['DOI(s)'] = "https://doi.org/10.1051/0004-6361/201323238"

gattano2014['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#gattano2014.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#gattano2014 = gattano2014[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#gattano2014.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

#gattano2014
# Here we are stripping out any objects previously listed by Veron-Cetty+2010 because we already have all of those.
# Gattano+ did not provide coordinates, so they are not being cited for having recovered those objects. 
# Sorry, not sorry. Do your goddamn job right. They'll be lucky if we cite them for the remaining objects.
gattano2014 = gattano2014[~gattano2014['Ref1'].str.contains('M')]
gattano2014 = gattano2014[~gattano2014['Ref2'].str.contains('M')]

gattano2014.reset_index(drop=True, inplace=True)
gattano2014.drop(5, inplace=True)
gattano2014.reset_index(drop=True, inplace=True)


#gattano2014['Name1'] = str(gattano2014['LQAC'])+str(gattano2014['QSO1'])
#gattano2014['Name2'] = str(gattano2014['LQAC'])+str(gattano2014['QSO2'])

#print(len(gattano2014))

#gattano2014 = gattano2014[~gattano2014['Ref1'].str.contains('M')]
#gattano2014 = gattano2014[~gattano2014['Ref2'].str.contains('M')]
#print(len(gattano2014))
#

#gattano2014t2 = ((Table.read('Tables/Gattano2014/J_A+A_583_A75/lqac3.dat', readme = 'Tables/Gattano2014/J_A+A_583_A75/ReadMe', format='ascii.cds')).to_pandas()) 


#for col in gattano2014t2.columns:
#    print(col)

#gatnames1 = ['LQAC_052-000_030','LQAC_253+026_011','LQAC_171+003_009','LQAC_122+031_018','LQAC_238+004_003',\
#             'LQAC_238+004_003','LQAC_137+000_025','LQAC_179+028_004','LQAC_230+056_004','LQAC_250+039_019',\
#             'LQAC_144+033_040','LQAC_136+000_002','LQAC_143+033_020','LQAC_173+008_005']
##gatnames2 = gattano2014['Name2'].to_list()
#
#for index, row in gattano2014t2.iterrows():
#    if row['LQAC'] in gatnames1:
#        print(row['RAdeg'],row['DEdeg'],row['LQAC'])
#
##for index, row in gattano2014t2.iterrows():
##    if row['LQAC'] in gatnames2:
##        print(row['RAdeg'],row['DEdeg'])
#
#gatnames2 = ['LQAC_052-000_031','LQAC_253+026_012','LQAC_171+003_010','LQAC_122+031_019','LQAC_238+004_004',\
#             'LQAC_238+004_004','LQAC_137+000_026','LQAC_179+028_005','LQAC_230+056_005','LQAC_250+039_020',\
#             'LQAC_144+033_041','LQAC_136+000_003','LQAC_143+033_021','LQAC_173+008_006']
#
#for index, row in gattano2014t2.iterrows():
#    if row['LQAC'] in gatnames2:
#        print(row['RAdeg'],row['DEdeg'],row['LQAC'])


In [48]:
# COORDINATES FOR 52.6 -0.29, 143.5 33.2, AND 171.7 3.97 COME FROM THE SECOND OBJECT; AGN1 COORDINATES ARE NOT \
# UNIQUE AND ARE COPIES OF AGN2

# COORDINATES FOR 122.4 31.4, 144.7 33.5, AD 179.3 28.9 COME FROM AGN 1 AND NOT FROM AGN2. WE USE IT ONLY FOR MATCHING

#52.63554575,-0.295480679,52.63554575,-0.295480679
#122.455014528,31.457583586,122.455014528,31.457583586
#136.074754543,0.696735558,136.079662361,0.701409223
#137.346376068,0.03444991,137.350046448,0.036406017
#143.589499821,33.29147088,143.589499821,33.29147088
#144.774153892,33.597664282,144.774153892,33.597664282
#171.781967448,3.973639181,171.781967448,3.973639181
#173.740569312,8.826454287,173.747401997,8.823148192
#179.316362001,28.900414404,179.316362001,28.900414404
#230.282964366,56.413459883,230.292168016,56.409998785
#238.074757379,4.946330492,238.07537404,4.943132964
#250.707794,39.397465,250.712790503,39.402494685
#253.75546703,26.088182747,253.758407145,26.08791982

# Order of the table
#LQAC_052-000_030/031
#LQAC_253+026_011/012
#LQAC_171+003_009/010
#LQAC_122+031_018/019
#LQAC_238+004_003/004
#LQAC_238+004_003/004
#LQAC_137+000_025/026
#LQAC_179+028_004/005
#LQAC_230+056_004/005
#LQAC_250+039_019/020
#LQAC_144+033_040/041
#LQAC_136+000_002/003
#LQAC_143+033_020/021
#LQAC_173+008_005/006

#52.63554575 -0.295480679 LQAC_052-000_031
#253.75546703 26.088182747 LQAC_253+026_011
#171.781967448 3.973639181 LQAC_171+003_010
#122.455014528 31.457583586 LQAC_122+031_018
#238.074757379 4.946330492 LQAC_238+004_003
#137.346376068 0.03444991 LQAC_137+000_025
#179.316362001 28.900414404 LQAC_179+028_004
#230.282964366 56.413459883 LQAC_230+056_004
#250.707794 39.397465 LQAC_250+039_019
#144.774153892 33.597664282 LQAC_144+033_040
#136.074754543 0.696735558 LQAC_136+000_002
#143.589499821 33.29147088 LQAC_143+033_021
#173.740569312 8.826454287 LQAC_173+008_005


data = """
52.63554575,-0.295480679,52.63554575,-0.295480679,LQAC_052-000_030,LQAC_052-000_030_031
253.75546703,26.088182747,253.758407145,26.08791982,LQAC_253+026_011,LQAC_253+026_011_012
171.781967448,3.973639181,171.781967448,3.973639181,LQAC_171+003_009,LQAC_171+003_009_010
122.455014528,31.457583586,122.455014528,31.457583586,LQAC_122+031_018,LQAC_122+031_018_019
238.074757379,4.946330492,238.07537404,4.943132964,LQAC_238+004_003,LQAC_238+004_003_004
137.346376068,0.03444991,137.350046448,0.036406017,LQAC_137+000_025,LQAC_137+000_025_026
179.316362001,28.900414404,179.316362001,28.900414404,LQAC_179+028_004,LQAC_179+028_004_005
230.282964366,56.413459883,230.292168016,56.409998785,LQAC_230+056_004,LQAC_230+056_004_005
250.707794,39.397465,250.712790503,39.402494685,LQAC_250+039_019,LQAC_250+039_019_020
144.774153892,33.597664282,144.774153892,33.597664282,LQAC_144+033_040,LQAC_144+033_040_041
136.074754543,0.696735558,136.079662361,0.701409223,LQAC_136+000_002,LQAC_136+000_002_003
143.589499821,33.29147088,143.589499821,33.29147088,LQAC_143+033_020,LQAC_143+033_020_021
173.740569312,8.826454287,173.747401997,8.823148192,LQAC_173+008_005,LQAC_173+008_005_006
"""

# Convert the string data into a list of rows
rows = [line.split(',') for line in data.strip().split('\n')]

# Split and assign the values correctly
processed_rows = []
for row in rows:
    processed_row = list(map(float, row[:4])) + row[4:]
    processed_rows.append(processed_row)

# Create a pandas DataFrame
columns = ['RA1_deg', 'Dec1_deg', 'RA2_deg', 'Dec2_deg', 'Name1', 'Name2']
df = pd.DataFrame(processed_rows, columns=columns)

#print(df)

gattano2014 = pd.concat([gattano2014,df], axis=1)
#gattano2014

# converting the degree only coords to sexagesiaml for our table...

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=gattano2014['RA1_deg']*u.degree, dec=gattano2014['Dec1_deg']*u.degree, frame='icrs')
gattano2014['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
gattano2014['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)


gattano2014

,LQAC,QSO1/2,U1mag,U2mag,theta,z1,z2,Ref1,Ref2,SeqU,Seqz,QSO1,---,QSO2,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Name1,Name2,RA1,Dec1
0,LQAC_,052-000_030/031,24.52,19.53,2.8899,0.610,0.609,----E--------,----E-----KL-,88,107,052-000_030,/,052-000_031,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.8899,185.771961,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,52.635546,-0.295481,52.635546,-0.295481,LQAC_052-000_030,LQAC_052-000_030_031,3:30:32.53,-0:17:43.73
1,LQAC_,253+026_011/012,18.92,18.39,9.6202,1.904,1.899,----E----JKL-,----E-----KL-,37,48,253+026_011,/,253+026_012,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,9.6202,515.250350,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,253.755467,26.088183,253.758407,26.087920,LQAC_253+026_011,LQAC_253+026_011_012,16:55:01.31,26:05:17.46
2,LQAC_,171+003_009/010,23.92,17.38,10.8570,0.561,0.545,----E--------,----E----JKL-,10,114,171+003_009,/,171+003_010,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,10.8570,3080.407633,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,171.781967,3.973639,171.781967,3.973639,LQAC_171+003_009,LQAC_171+003_009_010,11:27:07.67,3:58:25.10
3,LQAC_,122+031_018/019,19.74,21.43,11.3030,1.300,1.301,----E-----KL-,----E-----KL-,108,96,122+031_018,/,122+031_019,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,11.3030,-129.971739,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,122.455015,31.457584,122.455015,31.457584,LQAC_122+031_018,LQAC_122+031_018_019,8:09:49.20,31:27:27.30
4,LQAC_,238+004_003/004,19.55,19.14,11.6820,1.564,1.565,----E-----KL-,----E-----KL-,60,77,238+004_003,/,238+004_004,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,11.6820,-116.591924,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,238.074757,4.946330,238.075374,4.943133,LQAC_238+004_003,LQAC_238+004_003_004,15:52:17.94,4:56:46.79
5,LQAC_,137+000_025/026,20.31,16.88,14.9780,1.887,1.873,----E-----KL-,----E----JKL-,5,53,137+000_025,/,137+000_026,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,14.9780,1453.463636,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,137.346376,0.034450,137.350046,0.036406,LQAC_137+000_025,LQAC_137+000_025_026,9:09:23.13,0:02:04.02
6,LQAC_,179+028_004/005,18.25,21.97,20.9990,0.349,0.327,----E----JKL-,----E-----KL-,28,128,179+028_004,/,179+028_005,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,20.9990,4915.960712,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,179.316362,28.900414,179.316362,28.900414,LQAC_179+028_004,LQAC_179+028_004_005,11:57:15.93,28:54:01.49
7,LQAC_,230+056_004/005,19.86,20.03,22.0590,1.580,1.570,----E-----KL-,----E-----KL-,113,76,230+056_004,/,230+056_005,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,22.0590,1161.160671,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,,230.282964,56.413460,230.292168,56.409999,LQAC_230+056_004,LQAC_230+056_004_005,15:21:07.91,56:24:48.46
8,LQAC_,250+039_019/020,0.00,19.81,22.8420,2.381,2.382,--------IJKL-,----E-----KL-,112,11,250+039_019,/,250+039_020,J2000,Optical,-99,J2000,Optical,-99,Dual AGN Candidate,

In [49]:
# Here we will match to the_whills, add Gattano+'s bib information, and then we'll concatenate the tables together
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(the_whills,gattano2014,100)

print(len(tmatches_1),len(tmatches_2))
# I've searched for matches out to even 300' since we do not have coordinates for one AGN or the other in 6 pairs \
# from Gattano+2014

for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Gattano+2014'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2014A&A...564A.117G' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/201323238'

#for i in the_whills['Paper(s)']:
#    if 'Gattano+2014' in i:
#        print('True')

gattano2014.drop(idx2, inplace=True)
gattano2014.reset_index(drop=True)

the_whills = pd.concat([the_whills,gattano2014]).reset_index(drop=True)

# verified that this all now works and appears to match everything up nicely...


4 4


In [50]:
# Now loading in the newly discovered 54 binary quasars presented in Findlay+2018:

findlay2018 = pd.read_csv('Tables/Findlay2018/Findlay_2018.csv', sep=',')

findlay2018['Name1'] = findlay2018['QSO_fg']
findlay2018['Name2'] = findlay2018['QSO_bg']

findlay2018['z1'] = findlay2018['z_fg']
findlay2018['z2'] = findlay2018['z_bg']
findlay2018['z1_type'] = "spec"
findlay2018['z2_type'] = "spec"



# Converting the coordinates
coordconvert = SkyCoord(ra = findlay2018['RA1'], dec = findlay2018['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
findlay2018['RA1_deg'] = coordconvert.ra.degree
findlay2018['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#findlay2018['RA2'] = findlay2018['RA']
#findlay2018['Dec2'] = findlay2018['Dec']

coordconvert = SkyCoord(ra = findlay2018['RA2'], dec = findlay2018['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
findlay2018['RA2_deg'] = coordconvert.ra.degree
findlay2018['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
findlay2018['Equinox1'] = "J2000"
findlay2018['Coordinate_waveband1'] = "Optical"
findlay2018['Coordinate_Source1'] = findlay2018['Survey']

findlay2018['Equinox2'] = "J2000"
findlay2018['Coordinate_waveband2'] = "Optical"
findlay2018['Coordinate_Source2'] = findlay2018['Survey']

findlay2018['System Type'] = 'Quasar Pair'

# Adding in some columns that we'll population via a Simbad or Ned search later
findlay2018['Brightness1'] = -100
findlay2018['Brightness_band1'] = -100
findlay2018['Brightness_type1'] = -100

findlay2018['Brightness2'] = -100
findlay2018['Brightness_band2'] = -100
findlay2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
findlay2018['Sep'] = findlay2018['theta\'']*60 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#findlay2018['Sep(kpc)'] = findlay2018['Sep']*((cosmo.arcsec_per_kpc_proper(findlay2018['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#findlay2018['delta_z'] = findlay2018['z']-findlay2018['z2']
#findlay2018['dV'] = (2.99e+5)*((1+findlay2018['z'])**2 - (1+findlay2018['z2'])**2)/((1+findlay2018['z'])**2+(1+findlay2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
findlay2018['Selection Method'] = "Optical Colors / Mid-IR Colors" #DPSELs
findlay2018['Confirmation Method'] = "Slit Optical Spectroscopy"
findlay2018['Paper(s)'] = "Findlay+2018"
findlay2018['BibCode(s)'] = "2018ApJS..236...44F"
findlay2018['DOI(s)'] = "https://doi.org/10.3847/1538-4365/aabee5"

findlay2018['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#findlay2018.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#findlay2018 = findlay2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#findlay2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']




findlay2018




,QSO_fg,RA1,Dec1,QSO_bg,RA2,Dec2,z_fg,z_bg,theta',R_bottom(pkpc),Survey,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J003308.63-083222.19,00:33:08.63,-08:32:22.19,J003307.31-083241.55,00:33:07.31,-08:32:41.55,3.038,3.043,0.459,216.433,SDSS,J003308.63-083222.19,J003307.31-083241.55,3.038,3.043,spec,spec,8.285958,-8.539497,8.280458,-8.544875,J2000,Optical,SDSS,J2000,Optical,SDSS,Quasar Pair,-100,-100,-100,-100,-100,-100,27.54,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
1,J012902.78+191824.46,01:29:02.78,+19:18:24.46,J012901.92+191847.18,01:29:01.92,+19:18:47.18,2.680,2.691,0.430,209.778,SDSS,J012902.78+191824.46,J012901.92+191847.18,2.680,2.691,spec,spec,22.261583,19.306794,22.258000,19.313106,J2000,Optical,SDSS,J2000,Optical,SDSS,Quasar Pair,-100,-100,-100,-100,-100,-100,25.80,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
2,J015415.22+032455.84,01:54:15.22,+03:24:55.84,J015416.43+032457.86,01:54:16.43,+03:24:57.86,2.660,3.219,0.304,148.640,SDSS,J015415.22+032455.84,J015416.43+032457.86,2.660,3.219,spec,spec,28.563417,3.415511,28.568458,3.416072,J2000,Optical,SDSS,J2000,Optical,SDSS,Quasar Pair,-100,-100,-100,-100,-100,-100,18.24,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
3,J022845.72-124643.92,02:28:45.72,-12:46:43.92,J022848.07-124706.78,02:28:48.07,-12:47:06.78,1.733,2.032,0.688,358.303,ATLAS,J022845.72-124643.92,J022848.07-124706.78,1.733,2.032,spec,spec,37.190500,-12.778867,37.200292,-12.785217,J2000,Optical,ATLAS,J2000,Optical,ATLAS,Quasar Pair,-100,-100,-100,-100,-100,-100,41.28,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
4,J023229.05-100123.48,02:32:29.05,-10:01:23.48,J023231.25-100102.92,02:32:31.25,-10:01:02.92,2.063,2.386,0.641,328.856,ATLAS,J023229.05-100123.48,J023231.25-100102.92,2.063,2.386,spec,spec,38.121042,-10.023189,38.130208,-10.017478,J2000,Optical,ATLAS,J2000,Optical,ATLAS,Quasar Pair,-100,-100,-100,-100,-100,-100,38.46,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
5,J031855.31-103040.30,03:18:55.31,-10:30:40.30,J031853.87-102945.32,03:18:53.87,-10:29:45.32,2.226,2.417,0.982,498.485,ATLAS,J031855.31-103040.30,J031853.87-102945.32,2.226,2.417,spec,spec,49.730458,-10.511194,49.724458,-10.495922,J2000,Optical,ATLAS,J2000,Optical,ATLAS,Quasar Pair,-100,-100,-100,-100,-100,-100,58.92,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
6,J032926.40-134732.22,03:29:26.40,-13:47:32.22,J032926.04-134831.51,03:29:26.04,-13:48:31.51,2.073,2.372,0.992,508.701,ATLAS,J032926.40-134732.22,J032926.04-134831.51,2.073,2.372,spec,spec,52.360000,-13.792283,52.358500,-13.808753,J2000,Optical,ATLAS,J2000,Optical,ATLAS,Quasar Pair,-100,-100,-100,-100,-100,-100,59.52,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5,
7,J033347.40-133928.44,03:33:47.40,-13:39:28.44,J033345.40-133938.41,03:33:45.40,-13:39:38.41,2.230,2.679,0.513,260.495,ATLAS,J033347.40-133928.44,J033345.40-133938.41,2.230,2.679,spec,spec,53.447500,-13.657900,53.439167,-13.660669,J2000,Optical,ATLAS,J2000,Optical,ATLAS,Quasar Pair,-100,-100,-100,-100,-100,-100,30.78,Optical Colors / Mid-IR Colors,Slit Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-

In [51]:
# Here we're matching against the_whills
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(the_whills,findlay2018,20)

print(len(tmatches_1),len(tmatches_2))

the_whills = pd.concat([the_whills,findlay2018])
# verified that are currently no matches between the Findlay2018 and the_whills


0 0


In [52]:
#the_whills

In [53]:
# now formatting the table and saving..
the_whills.drop(labels=['Table_flag','Notes','zph1l','zph1m','zph1u','zsp1','f_zsp1','zph2l','zph2m','zph2u',\
                        'zsp2','n_zsp2','f_Sep','[HMS2010]_1','umag_1','gmag_1','u_rmag_1','rmag_1','imag_1',\
                        'zmag_1','chi2s_1','chi2p_1','e_z_1','DelV_1','DelT_1','RT_1','[HMS2010]_2','umag_2',\
                        'gmag_2','u_rmag_2','rmag_2','imag_2','zmag_2','chi2s_2','chi2p_2','e_z_2','DelV_2',\
                        'DelT_2','RT_2','zem1','Sep1','zem2','LQAC','QSO1/2','U1mag','U2mag',\
                        'theta','Ref1','Ref2','SeqU','Seqz','QSO1','---','QSO2','QSO_fg','QSO_bg','z_fg','z_bg',\
                        'theta\'','R_bottom(pkpc)','Survey','level_0','index'], axis=1, inplace=True)



In [54]:
#the_whills

In [55]:
# saving..
the_whills.to_csv('Binary_quasars_tables_DR1.csv', sep=',', index=False)


# verified, we just need to go back and add naming conventions to the Tytler2009 targets!


In [56]:
# # Now we're going to work on matching the catalogs of Kirman+2008 and Tytler+2009 to remove duplicate entries

# kirk = SkyCoord(ra=kirkman2008_master['RA1_deg']*u.degree, dec=kirkman2008_master['Dec1_deg']*u.degree) # Storing the coordinates from Kirkman+2008
# tytler = SkyCoord(ra=tytler2009_pairs_master['RA1_deg']*u.degree, dec=tytler2009_pairs_master['Dec1_deg']*u.degree) # Storing the coordinates from Tytler+2009
# idx, d2d, d3d = kirk.match_to_catalog_sky(tytler)   # Now matching the Kirkman+2008 catalog to Tytler+2009 catalog
# # idx are the indices in the Tytler catalog which are the closest matching rows to the Kirkman catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# kirk_matches = kirk[sep_constraint] # Applying the filter to matches in the kirkman catalog
# catalog_matches = tytler[idx[sep_constraint]] # Applying the filter to matches in the Tytler catalog

# tytler_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the Tytler catalog
# tytler_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
# tytler_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# tytler_matchesRA1['Dec1'] = catalog_matches.dec.degree

# # Now, since we're not sure if the components have been mixed up between the two catalogs, we need to rematch \
# # the Kirkman RA1 and Dec1 to the Tytler RA2 and Dec2

# kirk = SkyCoord(ra=kirkman2008_master['RA1_deg']*u.degree, dec=kirkman2008_master['Dec1_deg']*u.degree) # Storing the coordinates from Kirkman+2008
# tytler = SkyCoord(ra=tytler2009_pairs_master['RA2_deg']*u.degree, dec=tytler2009_pairs_master['Dec2_deg']*u.degree) # Storing the coordinates from Tytler+2009
# idx, d2d, d3d = kirk.match_to_catalog_sky(tytler)   # Now matching the Kirkman+2008 catalog to Tytler+2009 catalog
# # idx are the indices in the Tytler catalog which are the closest matching rows to the Kirkman catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# kirk_matches = kirk[sep_constraint] # Applying the filter to matches in the kirkman catalog
# catalog_matches = tytler[idx[sep_constraint]] # Applying the filter to matches in the Tytler catalog

# tytler_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the Tytler catalog
# tytler_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
# tytler_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# tytler_matchesRA2['Dec1'] = catalog_matches.dec.degree

# # Now we'll concatenate these two dataframes and use the resultant table to remove the matched indices from the \
# # Tytler+2009 catalog. We'll defer to Kirkman+2008 for the coordinates (a completeyl arbitrary choice), and we \
# # *should* just rematch all of the Kirkman and Tytler sources to Simbad and get new redshifts, since I've noticed \
# # the redshifts between Kirkman+2008 and Tytler+2009 can be discrepant. 

# frames = [tytler_matchesRA1,tytler_matchesRA2] 
# tytler_duplicates = (pd.concat(fran vbn     mes)).reset_index(drop=True) #Concatenating them here...

# # There's a total of 21 matches between the two catalogs...

# # Now removing from the Tytler+2009 catalog the sources that also appear in the Kirkman+2008 catalog by removing \
# # the indices listed in the tytler_duplicates table
# tytler2009_master = (tytler2009_pairs_master[~tytler2009_pairs_master['index'].isin(tytler_duplicates['idx'])]).reset_index(drop=True)

# #print(kirkman2008_master.columns)
# #print(tytler2009_master.columns)


In [57]:
# # Renaming the columns so that we have matching column names between these catalogs
# kirkman2008_master.columns=['z1', 'mag', 'n_mag', 'z2', 'mag2', 'n_mag2', 'Sep', 'RA1', 'Dec1',\
#                             'RA2', 'Dec2', 'Equinox', 'Coordinate_waveband', 'Coordinate_Source', \
#                             'Brightness_type1', 'Brightness_type2', 'delta_z', 'dV', 'Sep(kpc)', \
#                             'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg','Dec2_deg']

# tytler2009_master.columns=['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox', \
#                            'Coordinate_waveband', 'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)', \
#                            'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
#                            'Dec2_deg', 'index']        

# # Dropping the index column from tytler2009
# tytler2009_master.drop(labels=['index'], axis=1, inplace=True)
    
# # Rearranging the columns before concatenating
# kirkman2008_master = kirkman2008_master[['z1', 'Sep', 'RA1', 'Dec1', 'z2','RA2', 'Dec2', 'Equinox',\
#                                          'Coordinate_waveband', 'Coordinate_Source','delta_z', 'dV', 'Sep(kpc)', \
#                                          'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
#                                          'Dec2_deg','mag', 'n_mag','mag2', 'n_mag2','Brightness_type1', \
#                                          'Brightness_type2',]]

# tytler2009_master = tytler2009_master[['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox', \
#                                        'Coordinate_waveband', 'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)', \
#                                        'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
#                                        'Dec2_deg']]

# # Now concatenating these frames...
# frames=[kirkman2008_master,tytler2009_master]
# kirktyt = (pd.concat(frames)).reset_index(drop=True) #This is now our master frame for the Kirkman+2008 and Tytler+2009 catalogs
# # Duplicates at this point have been removed. 
# # I have NOTE, however, noted which ones overlapped between the two using the Paper(s), BibCode(s), or DOI(s) columns
# kirktyt['index'] = kirktyt.index #We'll use this to remove duplicates in a moment...


In [58]:
# kirktyt

In [59]:
# # Now we need to check or duplicates between the kirktyt master catalog and the Hennawi catalog

# henn = SkyCoord(ra=hennawi2006['RA1_deg']*u.degree, dec=hennawi2006['Dec1_deg']*u.degree) # Storing the coordinates from Hennawi+2006
# KT = SkyCoord(ra=kirktyt['RA1_deg']*u.degree, dec=kirktyt['Dec1_deg']*u.degree) # Storing the coordinates from kirktyt master catalog
# idx, d2d, d3d = henn.match_to_catalog_sky(KT)   # Now matching the Hennawi+2006 catalog to Kirktyt master catalog
# # idx are the indices in the kirktyt catalog which are the closest matching rows to the hennawi catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# henn_matches = henn[sep_constraint] # Applying the filter to matches in the hennawi catalog
# catalog_matches = KT[idx[sep_constraint]] # Applying the filter to matches in the kirktyt catalog

# KT_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
# KT_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
# KT_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# KT_matchesRA1['Dec1'] = catalog_matches.dec.degree

# ###

# henn = SkyCoord(ra=hennawi2006['RA1_deg']*u.degree, dec=hennawi2006['Dec1_deg']*u.degree) # Storing the coordinates from Hennawi+2006
# KT = SkyCoord(ra=kirktyt['RA2_deg']*u.degree, dec=kirktyt['Dec2_deg']*u.degree) # Storing the coordinates from kirktyt master catalog
# idx, d2d, d3d = henn.match_to_catalog_sky(KT)   # Now matching the Hennawi+2006 catalog to Kirktyt master catalog
# # idx are the indices in the kirktyt catalog which are the closest matching rows to the hennawi catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# henn_matches = henn[sep_constraint] # Applying the filter to matches in the hennawi catalog
# catalog_matches = KT[idx[sep_constraint]] # Applying the filter to matches in the kirktyt catalog

# KT_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
# KT_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
# KT_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# KT_matchesRA2['Dec1'] = catalog_matches.dec.degree

# # Concatenating these frames now
# frames = [KT_matchesRA1,KT_matchesRA2]
# KT_dups = (pd.concat(frames)).reset_index(drop=True)
# # There are a total of 10 matches between the Kirktyt master catalog and the hennawi+ catalog.
# # We will defer to the Hennawi+ catalog, and therefore, we're dropping the rows from the Kirktyt table

# # Now removing from the Tytler+2009 catalog the sources that also appear in the Kirkman+2008 catalog by removing \
# # the indices listed in the tytler_duplicates table
# kirktyt_master = (kirktyt[~kirktyt['index'].isin(KT_dups['idx'])]).reset_index(drop=True)
# kirktyt_master.drop(labels=['index'], axis=1, inplace=True) # Now dropping the index column as it is not needed



In [60]:
# kirktyt_master

In [61]:
# #Now adding in the missing columns that are needed for the Kirkman and Tytler pairs:
# kirktyt_master['System Type'] = "Binary Quasar"
# kirktyt_master['Name1'] = ("J"+(kirktyt_master['RA1'])+(kirktyt_master['Dec1'])).str.replace(':','') # Adopting names for the pairs using the provided RA and Decs.
# kirktyt_master['Name2'] = ("J"+(kirktyt_master['RA2'])+(kirktyt_master['Dec2'])).str.replace(':','') 
# kirktyt_master['z1_type'] = -100
# kirktyt_master['z2_type'] = -100
# kirktyt_master['Selection Method'] = -100
# kirktyt_master['Confirmation Method'] = -100

# # Now renaming the columns to a more uniform scheme where:
# # mag and mag2 --> Brightness1 and Brightness2
# # n_mag and n_mag2 --> Brightness_band1 and Brightness_band2 
# kirktyt_master.columns=['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox','Coordinate_waveband',\
#                         'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)','Paper(s)', 'BibCode(s)', 'DOI(s)',\
#                         'RA1_deg', 'Dec1_deg', 'RA2_deg','Dec2_deg', 'Brightness1', 'Brightness_band1',\
#                         'Brightness2', 'Brightness_band2','Brightness_type1','Brightness_type2', 'System Type',\
#                         'Name1', 'Name2', 'z1_type','z2_type', 'Selection Method', 'Confirmation Method']

# # Now rearranging the columns before we concatenate with the Hennawi+2006 catalog...

# kirktyt_master = kirktyt_master[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
#                          'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
#                          'Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
#                          'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
#                          'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]


# # NOTE: I NEED TO COME BACK AND FIX THE RA AND DEC VALUES SO THAT THE NAMING IS ACTUALLY CORRECT, RATHER THAN \
# # CONTINUING TO HAVE (E.G.) 01HR BE LISTED AS 1:... AND NOT 01:...

In [62]:
# kirktyt_master

In [63]:
# print(hennawi2006.columns)
# print(kirktyt_master.columns)

In [64]:
# # Now that the columns from Hennawi+2006 and the Kirkman+Tytler catalogs have common names and ordering (and \
# # all irrelevant columns and duplicates have been dropped), we will concatenate these tables:

# frames=[hennawi2006,kirktyt_master]
# hennkirktyt = (pd.concat(frames)).reset_index(drop=True) # Concatenating the tables here


In [65]:
#hennkirktyt

In [66]:
# # This cell is simply checking to ensure that there are no common rows between the Wang+2009 double-peak-selected \
# # targets and the targets assembled in the Hennawi/Kirkman/Tytler catalogs. There are no matches!

# HKT = SkyCoord(ra=hennkirktyt['RA1_deg']*u.degree, dec=hennkirktyt['Dec1_deg']*u.degree) # Storing the coordinates from hennkirktyt
# wang = SkyCoord(ra=wang2009['RA1_deg']*u.degree, dec=wang2009['Dec1_deg']*u.degree) # Storing the coordinates from wang2009 master catalog
# idx, d2d, d3d = HKT.match_to_catalog_sky(wang)   # Now matching the hennkirktyt catalog to wang2009 master catalog
# # idx are the indices in the wang2009 catalog which are the closest matching rows to the hennkirktyt catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# HKT_matches = HKT[sep_constraint] # Applying the filter to matches in the hennkirktyt catalog
# catalog_matches = wang[idx[sep_constraint]] # Applying the filter to matches in the wang2009 catalog

# wang_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the wang catalog
# wang_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
# wang_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# wang_matchesRA1['Dec1'] = catalog_matches.dec.degree

# ###

# HKT = SkyCoord(ra=hennkirktyt['RA2_deg']*u.degree, dec=hennkirktyt['Dec2_deg']*u.degree) # Storing the coordinates from hennkirktyt
# wang = SkyCoord(ra=wang2009['RA1_deg']*u.degree, dec=wang2009['Dec1_deg']*u.degree) # Storing the coordinates from wang2009 master catalog
# idx, d2d, d3d = HKT.match_to_catalog_sky(wang)   # Now matching the hennkirktyt catalog to wang2009 master catalog
# # idx are the indices in the wang2009 catalog which are the closest matching rows to the hennkirktyt catalog

# max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
# sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
# HKT_matches = HKT[sep_constraint] # Applying the filter to matches in the hennkirktyt catalog
# catalog_matches = wang[idx[sep_constraint]] # Applying the filter to matches in the wang2009 catalog

# wang_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the wang catalog
# wang_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
# wang_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
# wang_matchesRA2['Dec1'] = catalog_matches.dec.degree

# # Concatenating these frames now
# frames = [wang_matchesRA1,wang_matchesRA2]
# wang_dups = (pd.concat(frames)).reset_index(drop=True)

# wang_dups

In [67]:
# # Now concatenating the hennkirktyt and the wang catalog
# frames=[wang2009,hennkirktyt]
# HKTW = pd.concat(frames).reset_index(drop=True)


In [68]:
# # Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
# for index, row in tmatches.iterrows():
#     if row['Table_flag']!='Table2':
#         tmatches.at[index, 'Paper(s)'] += ' ; Smith+2012'
#         tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJ...752...63S' 
#         tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/752/1/63'

# # Now clipping out all Smith+2010 rows from the matches table
# tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# # Concatenating everything together to generate a master table here
# the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
# the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011d,3)

# #len(tmatches)

# # This finds 16 matches, which is exactly what we'd expect (there are 16 doubles in the table from Fu+2011)

# for i, j in zip(idx1, idx2):
#     #print("i:", i, "j:", j)
#     #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
#     #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
#     the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
#     the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
#     the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
#     the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
#     the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
#     the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# for index, row in the_whills.iterrows():
#     if index in idx1:
#         the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
#         the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
#         the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
#         the_whills.at[index, 'Notes'] += ' Companion(s) within 3 arcseconds.'

